<div id='outline_algos_framework'> </div> 

# outline 

* [to do](#todo)
* [imports](#imports)
* Training pipeline and walk through 
    * [setup](#setup)
        * [save and load key objects](#HANDLING__SIGNAL_DICT__MODEL__FRAMEWORK_RESULTS__PIPELINE)
            * `signal_dict`, pytorch `model`s, `framework_results`, `gs_results`
        * [run parameters](#run_parameters)
            * create `one_run_params` from `grid_search_params` 
        * [feature and target <b> formation </b>](#target_and_feature_formation) 
            * create `prices` DataFrame from trade level data in clickhouse
            * crypto quant data processing
            * [minute granularity info from `prices`](#minute_granularity_features_from_prices)
                * features created using `make_feature_set()` which is defined via `print_feature_making_code(desired_features_dict)`
            * [avellaneda stoikov](#avellaneda_stoikov_mm) feature creation
        * [feature and target <b> ingestion </b>](#target_and_feature_ingestion) - bring in features / targets from existing files
        * [check `prices` and `x` made or read in above](#check_prices_and_features)
        * [example target production](#example_target_production)
            * plot targets / new expirmental stuff with workarounds in `plot_framework()`
    * [separate model run, each part done individually](#separate_model_run) 
        * [model training and prediction](#model_train_and_preds)
        * [decision algorithm](#decision_algorithm) 
            * $g(f(x), prices) = action$
            * given model output of model trained on a rolling basis, $\hat{Y} \in \mathbb{R} ~[-1, 1]$ and the price history $P$ figure out what the transactions would have been. 
            * plotting the value of a portfolio
    * [grid search](#grid_search) over [targets, model, decision] params
        * prior gridsearch runs read in [setup](#setup)'s [run parameters](#run_parameters) section
        * set current run's `gridsearch_params`
    * [one model run](#one_model_run) runs once off [targets, model, decision] params
        * run whole process off `one_run_params` created in [setup](#setup)'s [run parameters](#run_parameters) section
    * [neural net DEV](#neural_net_dev)
        * [MLP](#neural_net_mlp)
            * [batched](#mlp_batched)
            * [rolling](#mlp_rolling) 
        * [LSTM](#neural_net_lstm)
* [**production** notes on how things are currently ran](#production_instruction) 
* [**future** development and live functionality ](#live_functionality) development --- there is a living suboutline maintained in this section
    [continious live development scratch space](#live_dev_scratch_space)
* [avellaneda stoikov](#avellaneda_stoikov_mm)
* [copy pasta](#copy_pasta) 
    * [sql console](#sql_console_copypasta)

#### [deving_here](#hereherehere)
<div id=''> </div>

 

<div id='todo'> </div>

# to do

* store this clickhouse command for remote access somewhere
    * `clickhouse-client --host=76.124.240.4 --port=8123 --user=default --password`

* go through this list and clean out trash, lots of things that are no longer relevant
* /algos/machine_specific/config_machine_specific.py should be merged into /algos/config.py, kindof big so do later
    * security check, everything machine specific should be isolated to the .env file. 
* check dot_env usage, with new init_<object> approach out of utils shouldn't need it all over 
* get a good clickhouse GUI 
* annual study of threshold based strategy 
* 
* # dockerize [update_trading_summary, update_signal, live_framework]
    * update_trading_summary
        * 
    * update_signal
        * 
    * live_framework
* # dashboard
* # add liquidation maps 
    * transactions (pushed to clickhouse for every trade executed (just do literally every trade for now if they happen in multiple, because they are small I doubt they would 
* add port value from positions table in plotting_dashboard.py
    * 
* `def make_vwap_dict(feature_params']`
* add `p_time_held` to `gs_df` 
    * this is to include in potential ranking metrics. 
* in the decision algorithm make a new switch for `d_signal` the derivative of signal 
    * if the signal is slowing down 
* in update `update_trading_summary.py`
    * make multi ticker... currently it is work,ing for BTC-USDT, add BTC-TUSD and ETH-TUSD 
    * if there is a long period of time missing (i.e. more than a day)
        * get the trades day by day to convert to trading summary and handle from there... too much data read to do a month of trades if needed for whatever reason (adding a new ticker for example) 
* for desired feature dict, I think its finally required to make a function that actually writes the code for calculations to its own file for doing. 
    * `from algos.make_feature_set_printed_fn import make_feature_set_printed_fn ` 
    * SEE COMMENT: # ###PAUL TODO: this should be depricated by code which prints a python file that does the calculations 
    * WITH THAT BEING SAID, this would only be to support different `desired_feature_dicts`... this would not be for speeding up calculations at each iteration
* reconcilliation of closed orders
* get new signal working from dev2... to do the `state_dict` --> \[`state` and `data`\] dicts mentioned below, going to be a hell of alot easier to spot errors and what not with a new signal running. This will allow a fresh initialization of a `state_dict` for a strategy so that we can see what isn't included anymore
    * 
* split `state_dict` removing all non-jsonable objects and putting them in  `data_dict`, which the goal of would be to only have data indexed in it  
* add multi asset support to state_dict... will need to be extended to `data_dict` 
    * `state_dict['bullish']` --> `state_dict[ticker]['bullish']`
* on `state_dict['order_open_dict']`, this should be checked in the live bot, it seems that some very old orders are sticking around, new ones are getting deleted... 
* update model / signal generation utilizing grid search data.  
* when ordering print out the state dictionary 
* add a "time of day" feature which should go with the most granular data 
    * don't think this will be terribly useful now but maybe absolutley necessary when adding non 24/7 market data 
    * one liner for this: 
        * `df['time_scaled'] = (x['timestamp'].dt.hour + x['timestamp'].dt.minute/60) / 24`
* `feature_params`
    * merging data of multiple time scales and data sources 
        * need a new method for this, the existing one is unreasonably slow.. maybe best to use a numpy array.. its as simple as forward filling less granular observations till there is another one
    * try to optimize compatibality with `one_run_params['reatures']`
        * big to do to get done here (though since getting trade only based model off the ground first this is less concerning) 
* `ch_client` ---- utils should rely on fewer `ch_client`s
    * generating one each function call is not remotely efficient, but doing this requires process mapping 
* order generation / afiguring still needs to be adjusted for ###PAUL_usd_denomination 
    * its working on `state_dict['bag_desired'][pair]['base_in_quote']` when it should probably be using a combination of newly added value representations 
* `update_signal` dockerized --  volume issue fixed
* #### in `live_framework_v2.py` 
    * clean up initialization of `state_dict` and filling of `params`, put them in their own wrapper functions
    * on <b>order tracking </b> via `open_orders.csv`
        * currently have two open_orders.csv written to disk. need to figure out if both are being used
        * in `/data/live/ports/<port_name>/open_orders.csv`
        * and `/data/live/ports/<port_name>/orders/<exchange>/open_orders.csv`
            * WE WANT THIS SECOND ONE ^^^^^^^^ 
        * see line ---- `state_dict['order_open_dict'] = {}`
    * params --> state_dict migration 
    * `pair_info_df` should go from params --> state_dict 
        * allowing for client generation to be done much further down improving flow of things 
        * need an `update_pair_info_df()` if it will be stored in the `state_dict`
    * on <b>docker</b>izing
        * regarding open orders (see line ---- state_dict['order_open_dict'] = {}   contains all open orders, backed up live in  ./data/<port_name>/open_orders.csv 
            * figure out way to best handle that (ideally state_dict can be saved as a json, could also use a pickle)
            * worst case read it from the file
            * FOR NOW PICKLE `state_dict` EACH TIME... ITS THE FASTEST WAY TO GET LIVE then do JSON
    * <b> ^^^^^^^^^^^^^ ALL THIS BEFORE DOCKERIZING ^^^^^^^^^^^^^ </b> 
    * loging in the live bot 
    * add print out of the decision state dict 
    
* #### live plotting. 
    * either `live_plot_framework()`
    * or include in existing one and add arg `mode='live'` or `'backtest'`

* get teams notifications going 

* `one_run_params['decision']  
    * this will be needed because the script `decision_on_signal.py` should be given a `decision_params` (or just a `one_run_params`?) 

* clickhouse trading summary
    * in `push_trading_summary_to_clickhouse()` make sure that duplicate rows aren't being pushed (probably best to make a parameter `overwrite` defaulted to `True`) to 

* #### new data sources
    * SP500 
    * open source <b> bloomberg like </b> data source (there was some python thing)          
* <b> FEATURE SET ADDITIONS </b>
    * ratio of two EWM prices 
    * SP500, gold, broader commodities... 
    * <b> volume kernels </b> (5m, 1h, 4h, 1d, 1w, 1m) 
        * dont be concerned with efficiency... this will be difficult. Make it efficient after it works  
        * kernels must be centered at the current price (otherwise it make no sense), SO..... 
            * to do this break the volume into price range bins that are much smaller than what then the final kernels to be used. Only significant issue will be on the shorter time scales if a mid price actually falls far from an interval. 
            * a way around this could be to give a "metric" for how far into the kernels above abnd below the current price we are (I don't like this though as there doesn't seem to be any value here having the data fed in non zero meaned then telling it where it is, just more complex space to learn. 
            * probably just do a grid +/-15% grid with 1% spacing 

* review how targets should be wrapped in a postprocessing function that directly outputs the product of `make_targets()` 
* ways to improve preformance in <b> rolling train </b>
    * adaptive number of epochs based on loss
    * roll in smaller steps, doing a triangled epoch setup (1 for 3 days ago, 2 for 2 days ago, 1 for 1 day ago, looping over a triangle, smallest last
    * save loss of training in a few ways 
        * list of lists per rolling iter 
        * dataframe of ['start_date', 'end_date', 'start_loss', end_loss'] 
    * to handle changing regieme
        * in rolling training keep going while the loss is going down by a certain ammount 

* investigate John's PNL function (should be in algos/decision.py) 
* clean up writing positions table 
* in decision algorithm, using `pred` instead of `signal` still 
* async stuff
    * make `update_trading_summaries` async, this is the lowest running task thats in a docker container. 
    * then move up to `update_signal` 
* database flow improvement
    * push [trades, trading_summary, signal] to clickhouse could all be put into one function. Most of it is the same
        * same
            * input: [exchange, symbol, overwrite_mode, existing_series, ]
        *  different
            * input [series_identifier] 
                * trades ---- trade_id... still select by a date range but then filter by trade id 
                * trading_summary ---- exchange, symbol, timestamp
                * signal ---- signal_id, timestamp
* docker ---> dont run as root 

<div id='imports'> </div>

# imports

In [ ]:
%load_ext autoreload

In [ ]:
# %autoreload 

import dotenv
# ### local imports ####PAUL TODO: clean up the sys path insertions 
import sys
import warnings

sys.path.insert(0, '..')

warnings.filterwarnings('ignore')

# ###PAUL TODO: i feel like there maybe some runtime issues having params imported, not sure though 
from algos.config import params

from algos.decision import (make_requests_dict,
                            run_decision_algo,
                            backtest_decision_making,
                            get_port_value_and_return_series,
                            get_sharpe_and_sortino,
                            analyze_returns_at_start_and_end_of_threshold_break,
                            get_feisable_transacts_list_from_time_strat,
                            make_transacts_dict_from_list,
                            run_simple_signal_threshold_and_hold_time_based_framework,)

from algos.feature import ( print_feature_making_code,
                            make_feature_df_from_results_of_printed_code,
                            zip_hour_and_min_level_df, 
                            rolling_normalize_feature_df,get_multiasset_trading_summaries,
                            make_utc_time_based_column,
                            merge_dfs_with_index_granularity_matching,
                            make_utc_time_df,
                            make_eaors_trades_features, 
                            make_features, )

from algos.feature_low_level import (get_ratio_of_metrics_rolling_sums,
                                    # zscore,
                                    calculate_momentum,
                                    calc_macd,
                                    calc_rsi,
                                    calc_rolling_std,)

from algos.gridsearch import (grid_search_over_parameters,
                              make_results_df,
                              run_full_process,
                              analyze_framework_run,
                              seperate_gridsearch_iter_dict_to_one_run_params, )
from algos.model import (
# rolling_decision_tree_train,
                         make_mlp_dataloader, 
                         NeuralNetwork,
                         train, 
                         test,
                         make_predictions_for_dataloader,
                         make_predictions_for_dataloader_features_only,
                         reshape_preds,
                         rolling_train_and_backtest_mlp,
                         make_signal_dict_from_preds,)
from algos.plotting import plot_framework
from algos.targets import (make_targets,
                           split_train_test,
                           analyze_signal,
                           print_signal_analysis, )  # ###PAUL this sort of thing should be housed under algos.utils....
from algos.utils import (convert_date_format,
                         get_date_list,
                         convert_trades_df_to_trading_summary,
                         find_runs,
                         check_prices_for_gaps,
                         get_trades_data, 
                         make_and_get_trading_summary,
                         preprocess_data,
                         preprocessing,  
                         cut_two_pd_dti_objects_to_matching_dtis,
                         fill_missing_minutes,
                         fill_trading_summary_interpolating_missing_minutes,
                         fill_trading_summary,
                         check_minute_integrity,
                         create_trading_summary_table, 
                         push_trading_summary_to_clickhouse,
                         get_binance_data_zip_file,
                         query_trading_summary,
                         get_signal_id,
                         deduplicate_df_on_index_only,
                         get_secret,
) 


# ###PAUL TODO: this should be depricated by code which prints a python file that does the calculations 
from algos.make_feature_set_printed_fn import make_feature_set_printed_fn 
# ###PAUL TODO: this should be depricated by code which prints a python file that does the calculations 


import ccxt
from clickhouse_driver import Client as CH_Client
from copy import deepcopy
from collections import Counter
import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import itertools
import json 
import lttb
import math 
# import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from scipy import signal
import time
import torch 
from torch import nn

# THIS ISNT WORKING RIGHT NOW AND WE NEED IT TO BUT IGNORE FOR NOW 
# THIS ISNT WORKING RIGHT NOW AND WE NEED IT TO BUT IGNORE FOR NOW 
# THIS ISNT WORKING RIGHT NOW AND WE NEED IT TO BUT IGNORE FOR NOW 
# THIS ISNT WORKING RIGHT NOW AND WE NEED IT TO BUT IGNORE FOR NOW 
# THIS ISNT WORKING RIGHT NOW AND WE NEED IT TO BUT IGNORE FOR NOW 
# THIS ISNT WORKING RIGHT NOW AND WE NEED IT TO BUT IGNORE FOR NOW 
# THIS ISNT WORKING RIGHT NOW AND WE NEED IT TO BUT IGNORE FOR NOW 


# from torch.utils.data import DataLoaders

ch_client = CH_Client(host=get_secret('CH_ALGOS_DB_HOST'),
                      port=int(get_secret('CH_ALGOS_DB_PORT')),
                      user=get_secret('CH_ALGOS_DB_USER'),
                      password=get_secret('CH_ALGOS_DB_PASSWORD'), 
                      database=get_secret('CH_ALGOS_DB_DATABASE'))

print(f"done") 

In [ ]:
from pprint import pprint

# make ccxt clients 

In [ ]:
from ccxt.pro import binanceus
from ccxt import binanceus as binanceus_nonpro   # used for ad hoc shit, no async... 

# Connect to Binance
binanceus = binanceus({
    'apiKey': get_secret('BINANCE_DATA_1_PUBLIC'),
    'secret': get_secret('BINANCE_DATA_1_PRIVATE'),
})


binanceus_nonpro = binanceus_nonpro({
    'apiKey': get_secret('BINANCE_DATA_1_PUBLIC'),
    'secret': get_secret('BINANCE_DATA_1_PRIVATE'),
})

# non websocket method must use non pro 

In [ ]:
# Fetch recent trades for a specific currency pair
symbol = 'KDA/USDT'
exchange = 'binance_us'
trades = binanceus_nonpro.fetch_trades(symbol)

# Print the trades
# for i, trade in enumerate(trades):
#     if 
#     print(trade)



# import  pprint
# pprint.pprint(trades[0]) 

# getting trades inserted into the database for a few different formats. 

In [ ]:
# pprint(trades)


In [ ]:
trade = trades[0]

trade_0 = {'timestamp': datetime.datetime.utcfromtimestamp(trade['timestamp']/1000),  # ###PAUL TODO: gon need to verify this 
           'exchange': exchange,
           'symbol': symbol,
             'id': trade['id'], 
             'price': trade['price'],
             'amount': trade['amount'],
             'side': 1 if trade['side'] == 'buy' else 0 
            }

trade_1 = {'timestamp': datetime.datetime.utcfromtimestamp(trade['timestamp']/1000),  # ###PAUL TODO: gon need to verify this 
           'exchange': exchange,
           'symbol': symbol,
             'id': trade['id'], 
             'price': trade['price'],
             'amount': trade['amount'],
             'side': 1 if trade['side'] == 'buy' else 0 
            }


In [ ]:
ch_client.execute('INSERT INTO algos_db.Trades VALUES ',
                  [trade_0, trade_1,], 
                  # positions_df.reset_index().to_dict('records'),
                  types_check=True,
                 )

In [ ]:
ch_client.query_dataframe("""SELECT *
FROM algos_db.Trades
ORDER BY timestamp DESC
LIMIT 30;  -- Adjust the number as needed""")


In [ ]:
trades_df = pd.DataFrame([trade_0, trade_1])

In [ ]:
trades_df.reset_index().to_dict('records')

In [ ]:
trades

In [ ]:
1701982151998

In [ ]:
datetime.datetime.utcfromtimestamp(trades[0]['timestamp']/1000)

# websocket method

In [ ]:
import asyncio
from pprint import pprint

# websocket. one exchange multi symobol

In [ ]:
async def process_trade_data(trades, ch_client, exchange):
    
    ch_formatted_trades = []
    print(len(trades)) 
    for trade in trades: 
        # trade = {'timestamp': datetime.datetime.utcfromtimestamp(int(trade['timestamp']/1000)),  # ###PAUL TODO: gon need to verify this 
        trade = {'timestamp': datetime.datetime.utcfromtimestamp(trade['timestamp']/1000),  # ###PAUL TODO: gon need to verify this 
                 'exchange': exchange,
                 'symbol': trade['symbol'],
                 'id': trade['id'], 
                 'price': trade['price'],
                 'amount': trade['amount'],
                 'side': 1 if trade['side'] == 'buy' else 0 
                }
        ch_formatted_trades.append(trade)

    ch_client.execute('INSERT INTO algos_db.Trades VALUES ',
                      ch_formatted_trades, 
                      types_check=True,
                     )
    
async def process_trades_periodically(processing_interval, ch_client):
    while True:
        if accumulated_trades:  # Check if there are trades to process
            await process_trade_data(accumulated_trades, ch_client, exchange='binance')
            accumulated_trades.clear()  # Clear the list after processing
        await asyncio.sleep(processing_interval)

# List to store accumulated trades
accumulated_trades = []

# Start processing task
processing_interval = 10  # Process trades every 10 seconds
processing_task = asyncio.create_task(process_trades_periodically(processing_interval, ch_client))

# Main loop to collect trades indefinitely
while True:
    trades = await binanceus.watch_trades_for_symbols(symbols=['BTC/USDT', 'KDA/USDT', 'ETH/USDT', 'LINK/USDT', 'ROSE/USDT'] )
    accumulated_trades.extend(trades)


In [ ]:
import asyncio
from collections import defaultdict

# Function to process trades for a specific exchange
async def process_trade_data(trades, ch_client, exchange):
    ch_formatted_trades = []
    print(f"trades collected for exchange: {exchange} ---> {len(trades)}")
    
    for trade in trades:
        formatted_trade = {
            'timestamp': datetime.datetime.utcfromtimestamp(trade['timestamp']/1000),
            'exchange': exchange,
            'symbol': trade['symbol'],
            'id': trade['id'], 
            'price': trade['price'],
            'amount': trade['amount'],
            'side': 1 if trade['side'] == 'buy' else 0
        }
        ch_formatted_trades.append(formatted_trade)
    
    ch_client.execute('INSERT INTO algos_db.Trades VALUES ', ch_formatted_trades, types_check=True)


# Function to collect trades for each exchange
async def collect_trades(exchange_client, exchange_name, symbols, trade_storage):
    while True:
        for symbol in symbols:
            trades = await exchange_client.watch_trades_for_symbols(symbols=symbol)
            trade_storage.extend([(exchange_name, trade) for trade in trades])
        await asyncio.sleep(1)  # Adjust as needed

# Process trades periodically
async def process_trades_periodically(processing_interval, ch_client, trade_storage):
    while True:
        if trade_storage:  # Check if there are trades to process
            for exchange, trade in trade_storage:
                await process_trade_data(trade, ch_client, exchange)
            trade_storage.clear()  # Clear the list after processing
        await asyncio.sleep(processing_interval)

# Exchange to symbols mapping
exchange_symbols = {
    'binance': ['BTC/USDT', 'KDA/USDT', 'ETH/USDT', 'LINK/USDT'],
    # Add other exchanges and their symbols as needed
}

# List to store accumulated trades
accumulated_trades = []

# Start trade collection tasks for each exchange
collection_tasks = []
for exchange, symbols in exchange_symbols.items():
    # Create an exchange client instance for each exchange
    exchange_client = create_exchange_client(exchange)  # Replace with actual client creation logic
    task = asyncio.create_task(collect_trades(exchange_client, exchange, symbols, accumulated_trades))
    collection_tasks.append(task)

# Start processing task
processing_interval = 10  # Process trades every 10 seconds
processing_task = asyncio.create_task(process_trades_periodically(processing_interval, ch_client, accumulated_trades))

# Run the event loop
loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.gather(*collection_tasks, processing_task))


# ### ATTEMPTING TO GET MULTIPL3E EXCHANGES AND TICKERS WORKING IN ONE COLLECTION SCRIPT

In [ ]:
import asyncio
from collections import defaultdict

async def collect_trades(exchange, symbol, trade_storage):
    while True:
        trades = await exchange.watch_trades(symbol)
        trade_storage[exchange.name][symbol].extend(trades)
        await asyncio.sleep(1)  # Adjust frequency as needed

async def process_trade_data(trades, ch_client):
    # Your existing processing logic here
    ...

async def process_trades_periodically(processing_interval, trade_storage, ch_client):
    while True:
        for exchange_name in trade_storage:
            for symbol in trade_storage[exchange_name]:
                if trade_storage[exchange_name][symbol]:
                    await process_trade_data(trade_storage[exchange_name][symbol], ch_client)
                    trade_storage[exchange_name][symbol].clear()
        await asyncio.sleep(processing_interval)

# Dictionary to store accumulated trades, organized by exchange and ticker
accumulated_trades = defaultdict(lambda: defaultdict(list))

# List of exchanges and their symbols
exchanges = [binanceus,] #  another_exchange]  # Replace with actual exchange objects
symbols = ['BTC/USDT', 'ETH/USDT']  # Replace with the list of tickers you're interested in

# Start trade collection tasks
collection_tasks = [asyncio.create_task(collect_trades(exchange, symbol, accumulated_trades)) 
                    for exchange in exchanges for symbol in symbols]

# Start processing task
processing_interval = 10  # Process trades every 10 seconds
processing_task = asyncio.create_task(process_trades_periodically(processing_interval, accumulated_trades, ch_client))

# Run the event loop
loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.gather(*collection_tasks, processing_task))


[back to top](#outline_algos_framework) 

<div id='setup'> </div>

# setup

In [ ]:
pip install ccxt

<div id='HANDLING__SIGNAL_DICT__MODEL__FRAMEWORK_RESULTS__PIPELINE'> </div> 

* ##### save and load key objects
    * [`target_series`](#loading_target_series)
    * [`signal_dict`](#loading_signal_dict)
    * [`framework_results`](#loading_framework_results)
    * [`gs_results`](#loading_gs_results)
    * [`one_run_params`](#loading_one_run_params)
    
    * ##### ###PAUL TODO: this should probably go in utils
        * but due to the recent need to reorganize keep in NB and interate untill confident that the `signal_dict`/`model` --> `framework_results` pipeline won't need significant adjustment

<div id='loading_target_series'> </div>

* ### save and load `target_series`

In [ ]:
# del signal_dict_fp

# # ### NAME OF `signal_dict` run
# #
# #
target_series_name = f"target_serires____2023_04_25____peaks_0_06____min_peak_dist_480____interval_len_14d____smooth_0_01"
target_series_fp = f"{params['dirs']['data_dir']}pickled_target_series/{target_series_name}.pickle"

# # ### SAVING
# #
# pickle.dump(target_series,  #   <<<------- COPY PASTE DESIRED VARIABLE HERE
#             open(target_series_fp, "wb"))

# # ### OPENING
# #
# with open(target_series_fp, 'rb') as f:
#      target_series = pickle.load(f) #   <<<------- COPY PASTE DESIRED VARIABLE HERE

# y = target_series # name y for work down low # ###PAUL TODO: variable name conflict

### LEAVE ON ALWAYS
#
del target_series_fp

<div id='loading_signal_dict'> </div>

* ### save and load `signal_dict`

In [ ]:

# # ### NAME OF `signal_dict` run
# #
# #
# signal_dict_name = f"signal_dict____2023_08_01____mlp_rolling____first_multi_asset_feature_set"  
# signal_dict_name = f"signal_dict____2023_08_03___mlp_rolling____validate"  
# signal_dict_name = f"signal_dict____2023_08_23___mlp_rolling____to_2023_07_18"
# signal_dict_name = f"signal_dict____2023_09_01___mlp_rolling_smaller_model"
signal_dict_name = f"signal_dict____2023_09_01___mlp_rolling____large_model"
# signal_dict_name = f"signal_dict____2023_09_01___mlp_rolling____large_model_bigger_learning_rate"
# signal_dict_name = f"signal_dict____2023_09_01___mlp_rolling____large_model_SGD_1e4"

signal_dict_fp = f"{params['dirs']['data_dir']}pickled_signal_dicts/{signal_dict_name}.pickle"

# # ### SAVING
# #
# if os.path.isfile(signal_dict_fp): 
#     print(f"signal already exists at that filepath") 
# else:
#     pickle.dump(signal_dict,  #   <<<------- COPY PASTE DESIRED VARIABLE HERE
#                 open(signal_dict_fp, "wb"))


# # ### OPENING
# #
with open(signal_dict_fp, 'rb') as f:
     signal_dict = pickle.load(f) #   <<<------- COPY PASTE DESIRED VARIABLE HERE

### LEAVE ON ALWAYS
#
del signal_dict_name; del signal_dict_fp

In [ ]:
signal_dict.keys()

<div id='loading_framework_results'> </div>

* ### save and load `framework_results`

In [ ]:
# # ### NAME OF `framework_results` 
#
framework_results_name = f"framework_results____0_06_peaks____MSE_loss____on_preds__0_0075_ewm__4_day_rolling_norm____split_0_65__0_15____NO_shuffled_train"
framework_results_fp = f"{params['dirs']['data_dir']}pickled_signal_dicts/{framework_results_name}.pickle"

# # ### SAVING
# #
# if os.path.isfile(framework_results_fp): 
#     print(f"FRAMEWORK RESULTS already exists at that filepath")
# else:
#     pickle.dump(framework_results,  #   <<<------- COPY PASTE DESIRED VARIABLE HERE
#                 open(framework_results_fp, "wb"))

# # ### OPENING
# #
# with open(framework_results_fp, 'rb') as f:
#      framework_results = pickle.load(f) #   <<<------- COPY PASTE DESIRED VARIABLE HERE
        
### LEAVE ON ALWAYS
#
del framework_results_name; del framework_results_fp

<div id='loading_gs_results'> </div>

* ### save and load `gs_results`

In [ ]:
# ### reading in prior grid searches
#

gs_file = f"grid_search____2023_08_01____mlp_rolling____first_multi_asset_feature_set"

gs_file = f"grid_search____2023_08_01____mlp_rolling____v2"  # 1
# gs_file = f"grid_search____2023_08_01____mlp_rolling____v3"  # 2
# gs_file = f"grid_search____2023_08_01____mlp_rolling____v4"    # 1
# gs_file = f"grid_search____2023_08_29____mlp_rolling_v2"
# gs_file = f"grid_search____2023_09_05____on____signal_dict____2023_09_01___mlp_rolling_smaller_model"
# gs_file = f"grid_search____2023_09_06____on____signal_dict____2023_09_01___mlp_rolling_smaller_model____any_two_set"

# gs_file = f"grid_search____most_recent.pickle" 

gs_fp = f"{params['dirs']['data_dir']}gridsearches/{gs_file}.pickle"

# # ### SAVING gridsearch
# # 
# if os.path.isfile(gs_fp): 
#     print(f"gs_df already exists at that filepath")
# else:
#     gs_df.to_pickle(gs_fp)

# # ### OPENING gridsearch 
# # 
gs_df = pd.read_pickle(gs_fp)

### LEAVE ON ALWAYS
#
del gs_file; del gs_fp


#
# ###
# ### getting columns we are interested in seeing the results of 
# ###
#
col_subset = []
for col in gs_df.columns:
    if 'decision_' in col: 
        col_subset.append(col) 
preformance_cols = ['sharpe', 'sortino', 'pnl', 'num_transacts', 'ranking_metric', ]
show_cols = col_subset + preformance_cols


gs_df['ranking_metric'] = gs_df['sharpe']**3 * gs_df['pnl'] / gs_df['num_transacts' ]**(2.5) *1_000_000

In [ ]:
gs_df.shape

In [ ]:
gs_df.sort_values(by='pnl').iloc[-10:][show_cols]

In [ ]:
gs_df.sort_values(by='pnl').iloc[-10:][show_cols]


# gs_df.sort_values(by='pnl').iloc[-10:][show_cols]
# gs_df.sort_values(by='ranking_metric')[show_cols]

In [ ]:
# decision_threshold	decision_pred_dist	decision_price_dist	decision_stop_limit		decision_to_neutral_threshold	decision_to_neutral_pred_dist	decision_to_neutral_price_dist	decision_to_neutral_stop_limit
np.sort(gs_df['decision_to_neutral_stop_limit'].unique())

* ##### <i> exploring `gs_results`  </i> ---- isolating observations with by a column pandas can't match by comparing equality via `==`  

In [ ]:
# cmp = lambda x: x==[]
cmp = lambda x: x==['threshold', 'pred_dist', 'price_dist']
mask_decision_to_neutral_any_two = gs_df['decision_to_neutral_any_two'].apply(cmp)

* ##### `one_run_params` derived from `gs_df`

In [ ]:
# ### getting one run params
#
#
gs_df['ranking_metric'] = gs_df['sharpe'] * gs_df['pnl'] / gs_df['num_transacts' ] 

one_run_params = seperate_gridsearch_iter_dict_to_one_run_params(gs_df.sort_values(by='ranking_metric').iloc[-1])
avg_best = gs_df.sort_values(by='ranking_metric').tail(100).mean()

for name, val in avg_best.iteritems():
    for param_type in ['feature_', 'targets_', 'model_', 'decision_', ]:
        param_no_underscore = param_type.replace("_", "") 
        if name.startswith(param_type):
            name = name.replace(param_type, "")
            # print(f"{type(one_run_params[param_no_underscore][name])}")
            if type(one_run_params[param_no_underscore][name]) in [np.float64, float]: 
                # print(f"name: {name} ---- param_type: {param_type} ---- param_no_type_prefix: {param_no_underscore}")
                one_run_params[param_no_underscore][name] = val
                
# # ### GET THE PARAMS FROM ONE INDIVIDUAL RUN ----  EX: just want to run the best model on a grid search 
# #
# #
one_run_params = seperate_gridsearch_iter_dict_to_one_run_params(gs_df.sort_values(by='pnl').iloc[-1])

[back to top](#outline_algos_framework) 


<div id='loading_one_run_params'> </div>

* ### save and load `one_run_params` 
    * # ###PAUL TODO: for now hard coding `desired_features_dict` but this is where I would like this to go for the flow of the notebook 

In [ ]:
# ### dict containing atleast one of every option
#
#
desired_features_dict = {"buy_to_sell_count_ratio": {"interval_lens": [1, 3, 10, 28, 59, 119, 360],
                                                     "function_name": "get_ratio_of_metrics_rolling_sums",
                                                     # note, if the argument is str it is written "'string'"
                                                     "kwargs": {"df": "trading_summary",
                                                                "metric_1": "'buyer_is_maker'",
                                                                "metric_2": "'buyer_is_taker'",
                                                                }
                                                     },

                         "buy_to_sell_vol_ratio": {"interval_lens": [1, 3, 10, 28, 59, 119, 360, 1440, ],
                                                   "function_name": "get_ratio_of_metrics_rolling_sums",
                                                   "kwargs": {"df": "trading_summary",
                                                              "metric_1": "'buy_base_vol'",
                                                              "metric_2": "'sell_base_vol'",
                                                              },
                                                   },

                         "momentum": {"interval_lens": [1, 3, 10, 28, 59, 119, 240, 1200, 4320, 14400],
                                      "function_name": "calculate_momentum",
                                      "kwargs": {"df": "trading_summary",
                                                 "col": "'vwap'",
                                                 },
                                      },

                         "rsi": {'interval_lens': [10, 28, 59, 240, 1200, 4320, 14400, 56000],
                                 'function_name': 'calc_rsi',
                                 'kwargs': {'series': "trading_summary['vwap']"},
                                 },

                         # ###PAUL this is not what is wanted... rolling variance is a good replacement for what this was trying to do 
                         # "roll_std_price": {'interval_lens': [15, 28, 59, 119, 240, 1200, 4320, 14400],
                         #                    'function_name': 'calc_rolling_std',
                         #                    'kwargs': {'series': "prices['vwap']"},
                                            # },
                         "roll_std_vol": {'interval_lens': [15, 28, 59, 119, 240, 1200, 4320, 14400],
                                          'function_name': 'calc_rolling_std',
                                          'kwargs': {'series': "prices['total_base_vol']"},
                                          },
                         
                         
                         # ###PAUL TODO: ROLLING STD OF VOLATILITY 
                         #
                         #

                         # more complex metrics below this line
                         # _=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_

                         'macd': {'function_name': 'calc_macd',
                                  'kwargs': {'series': "prices['vwap']",
                                             },
                                  'span_tuples': [(12, 26, 9), (4, 26, 9), (13, 26, 1), (12, 26, 3)],
                                  'multipliers': [1, 5, 15, 60, 150, 1600 ],
                                  },

                         # EXAMPLE: keep this feature_name it is skipped by the code writer if left as is
                         # _=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_=_
                         'feature_name': {'interval_lens': [30, 60, 120, ],
                                          'function_name': 'function_to_make_metric',
                                          'kwargs': {'col_name': 'input_str',
                                                     'metric_1': 'input_str',
                                                     'metric_2': 'input_str',
                                                     }
                                          },
                         }


# print_feature_making_code(desired_features_dict)

In [ ]:
feature_params = {
    # for live functionality what process historical EAORS trades sits between `update_trading_summary.py` 
    
    # ###PAUL TODO: consider refactoring sources to feature_params['sources'][source] 
    # 'sources': {                   dictionary continued below, but not used for now 
    
    
    # TODO: `eaors_trades` relies on the same pair only appearing once, this is unrealistic as we may want the same pair from two exchanges.
    # seems like the best approach would be [exchange][pair] ... with the note that this would not support spot vs futures. 
    # to future proof, it would be necessary to use ['eaors_trades']['spot/futures'][exchange][pair] then walk down that tree starting at `eaors_trades`
    
    
    # TODO: `max_data_mode` ---- have function get all inputs from as early as possible 
    # TODO: also provide a dictionary of how early each source was, the cut (total or broken down?) needed to process it and limiting feature
    'eaors_trades': { # 'data_source' : {source_specific_processing_information}, 
                      # ###PAUL TODO: with this structure there is no way to do cross exchange metrics, which could offer significant insight
                      # ###PAUL TODO: for example... binance_volume / kucoin_volume along with price....
                      # ###PAUL TODO:    - volume would have to be ratio of rolling sum, price could be the instaneous (
                      # ###PAUL TODO: regardless, this would be something that could be interesting 
                      # ###PAUL TODO: 
                      'BTC-USDT': {'desired_features_dict': desired_features_dict,
                                  'exchange': 'binance',
                                  'start_date': (2018, 1, 1), 
                                  'end_date': (2023, 7, 18), 
                                  'alternative_data_pair': None,
                                 },
                     'BTC-TUSD': {'desired_features_dict': desired_features_dict,
                                  'exchange': 'binance',
                                  'start_date': (2018, 1, 1), 
                                  'end_date': (2023, 7, 18), 
                                  'alternative_data_pair': 'BTC-USDT',
                                  'alternative_data_exchange': 'binance',
                                  'alternative_start_date': (2018, 1, 1), 
                                  'alternative_end_date': (2023, 2, 15), 
                                 },
                     'ETH-USDT': {'desired_features_dict': desired_features_dict,
                                  'exchange': 'binance',
                                  'start_date': (2018, 1, 1), 
                                  'end_date': (2023, 7, 18), 
                                  'alternative_data_pair': None,
                                 },
                     'ETH-TUSD': {'desired_features_dict': desired_features_dict,
                                  'exchange': 'binance',
                                  'start_date': (2018, 1, 1), 
                                  'end_date': (2023, 7, 18), 
                                  'alternative_data_pair': None,
                                  'alternative_data_exchange': 'binance',
                                  'alternative_start_date': (2018, 1, 1), 
                                  'alternative_end_date': (2023, 2, 15), 
                                 },                        
    },
    
    # 'eaors_orderbook': {}, 
    # 'l_map': {}, 
    # 'sp500': {},
    # 'commodities': {},
    # 'forex': {'DXY': {'desired_features_dict': desired_features_dict},  # could be nearly identical to EAORS trade pipeline 
    
    'utc_time': True,  
    
    # ###PAUL TODO: consider refactoring sources to feature_params['sources'][source] 
    #
    # }                  dictionary started above, but not used for now 
        
    'preprocess_rolling_norm_n_obvs' : 20 * 24 * 60,
    
}




target_params= \
    {
     'min_peak_dist': 480,
     'peak_prominence': 0.055,
     'interval_len_in_steps': 35*24*60,
     'pd_ewm_alpha': 0.01
    }

model_params = \
    {
        'train_days': 365,
        'step_size': 7,
        'f_window': 0,

        'num_epochs_for_train': 14,
        'num_epochs_when_rolling': 3,
        
        'batch_size': 24 * 60,

        # dataloader shuffle stuff (planned for batch training, but may want to add to rolling) 
        'shuffle_train': False,
        'shuffle_test': False,  # this should always be False, maybe dont give it an argument

        'volatility_normalization_for_targets': False,  # ###PAUL TODO: check this calculation, OFF for now
        'return_window_n': 3*60,         # REQUIRED IF `volatility_normalization_for_targets` == True
        'variance_window_n': 3*24*60,    # REQUIRED IF `volatility_normalization_for_targets` == True

        'normalize_targets': True,  # normalizes targets per each iteration BEFORE cutting them for training
        # 'optimizer': torch.optim.Adam(model.parameters(), lr=1e-3),  
        # MAYBE NEED TO ADD optimizer TO BE DONE AFTER MODEL CREATION.... 
        # WITH THAT BEING SAID IIT MAY MAKE SENSE TO ADD MODEL INITIALIZATION IN THESE PARAMS 
        # ^^^^^^^^^^^
    
        'loss_fn': nn.MSELoss(),
        
        # ### preds post processing parameters to make preds into a signal 
        'preds_pd_ewm_alpha': 0.01,
        'rolling_normalize_preds': True,
        'signal_norm_window': 5*24*60,   # REQUIRED IF `rolling_normalize_preds` == True
    }

decision_params = \
    {
        
    } 

one_run_params = \
    {'feature': feature_params,
      'target': target_params,
      'model': model_params,
      'decision': decision_params,
     }

# this belongs in params (i believe parts gets put in it, but after going multi asset it would be a good thing to try and do as 
# as close of a merger as possible between the notebook method and scripts building everything off of configs.py regardless) 

<div id='run_parameters'> </div>

## run parameters

In [ ]:
# ### notebook control variables
#
#
nb_params = \
{
    'read_trading_summary_from_db': True,
    
    # ###PAUL TODO: should be moved to `one_run_params['features']
    'make_min_features': True,
    'make_cq_features': False,
        
    'make_example_targets': False,
    'discresionary_signal': False,
    'single_run': True, 
    'gridsearch_run': False,
    'one_full_run': False,
    'live_trade_work': False, 
    'neural_net_dev': True, 
    'MLP_batch': False,
    'MLP_rolling': True,  
}


<div id='target_and_feature_formation'> </div>

## feature and target <b> formation </b>

* #### CREATE `trading_summary` from quering trades and summarizing
    * note `trading_summary` migrated to clickhouse which is now upated live but kept for example to help with understanding the pipeline 

In [ ]:
del run_once_make_features

In [ ]:
%%time

try:
    if run_once_make_features: 
        raise ValueError
except NameError:
    run_once_make_features = True 
    
x, max_cut_time = make_features(feature_params=feature_params, ch_client=ch_client)

In [ ]:
x.shape

In [ ]:
x.index

* ### ###PAUL TODO: `vwap_dict` is needed for 
    * target (live and backtesting)... though I think we can use trading_summaries. look into this 
    * backtesting
    * plotting 
    * ##### figure out the best place to put it 

In [ ]:
vwap_dict = get_multiasset_trading_summaries(feature_params['eaors_trades'], columns=['vwap'], ch_client=ch_client)

In [ ]:
vwap_dict

* ##### crypto quant data processing (currently depricated ---- keeping around because potential for revival) 

In [ ]:
# # ### CRYPTO QUIANT PROCESSING KEPT INCASE WE WANTED TO BRING THE DATA IN IN THE FUTURE (OR TO SEE IF IT HELPS) 
# #
# #
# if nb_params['make_cq_features']: 
#     crypto_quant_freq = "hour"
#     start_date = "2016-12-31"  # decent date to start general prune as many features NaN before this point 
#     ticker = "btc"
#     pct_nan = 0.05

#     # read in signal and info_df which contains metadata for columns in signal (api link, min / max date) 
#     signal_fp = f"{params['dirs']['data_dir']}crypto_quant/btc/signal_hour.pickle"
    
#     with open(signal_fp, 'rb') as f:
#         file = pickle.load(f)
#         hour_crypto_quant = file[ticker][0]
#         df_info = file[ticker][1]
#         hour_crypto_quant.index = hour_crypto_quant['date']  ###PAUL new add del_later
#         hour_crypto_quant.index = pd.to_datetime(hour_crypto_quant.index)
#         hour_crypto_quant.index.name = 'Timestamp'  # all pd.DTI's are named Timestamp

#         del hour_crypto_quant['date']
#         hour_crypto_quant.sort_values(by='Timestamp', inplace=True)

#     hour_crypto_quant = hour_crypto_quant[hour_crypto_quant.index >= start_date]
#     hour_crypto_quant_return = preprocessing(hour_crypto_quant, pct_nan, "return")
#     hour_crypto_quant_z_score = preprocessing(hour_crypto_quant, pct_nan, "z_score")

#     # add column name identifies for merge (makes it work easier too) 
#     hour_crypto_quant_return.columns = ['cq_return_' + col for col in hour_crypto_quant_return.columns]
#     hour_crypto_quant_z_score.columns = ['cq_zscore_' + col for col in hour_crypto_quant_z_score.columns]

#     hourly_feature_df = hour_crypto_quant_return.merge(right=hour_crypto_quant_z_score,
#                                                        how='outer',
#                                                        left_index=True,
#                                                        right_index=True,
#                                                       )

#     hourly_feature_df.shape
    
# print(f"done")

* #### find gaps in `prices` which represents gaps in trade level data

In [ ]:
info_on_missing_price_data = check_prices_for_gaps(x)

In [ ]:
x.head(1)

In [ ]:
x.tail(1)

In [ ]:
# ### ensure that prices contain every index within the features set 
# ### ALWAYS SHOULD RUN THIS CHECK 
#
#
trading_summary_range_of_features_mask = np.logical_and(min(x.index) <= trading_summary.index,  trading_summary.index <= max(x.index))
pd.testing.assert_index_equal(trading_summary[trading_summary_range_of_features_mask].index, x.index)
assert trading_summary[trading_summary_range_of_features_mask].index.equals(x.index)

<div id='example_target_production'> </div>

## example target production 
* due to the nature of the targets they need to be calculated at the time of training
* this is to explore how changes to target generation affect a good estimate of a real time implementable target series 

In [ ]:
vwap_dict['BTC-TUSD']

In [ ]:
nb_params['make_example_targets'] = True

In [ ]:
%%time

# nb_params['make_example_targets'] = True
if nb_params['make_example_targets'] == True:  # ###PAUL neural_net_dev shoulnd be removed when no longer needed  

    y, ideal_buy_sell_dict = make_targets(series=vwap_dict['BTC-TUSD']['vwap'], **one_run_params['target'] ) 
          
print(f"done")

In [ ]:
y

In [ ]:
if nb_params['make_example_targets'] == True:
    
    target_results = {'preds': y}   
    
    # ### PLOTTING SIGNAL
    #
    plot_requests = {
        'start_date': (2021, 9, 10),  # ###PAUL TODO: add start / end functionality to here 
        'end_date': (2021, 12, 10),
        # 'p_start': 0.95,
        # 'p_end': 1,

        'prices': True,
        'transact_times': False,
        'preds': False,
        'smoothed_preds': False,
        'signal': False, 
        'port_val_ts': False,
        'y_train_rti': True,
        'ideal_top_bottoms': False,
    }
    
    # temp plot dict
    plot_dict = {
        # 'signal': vwap_dict['BTC-TUSD'],
                 'preds': vwap_dict['BTC-TUSD'], 
    'y_train_rti': y, 
    }

    fig = plot_framework(plot_requests=plot_requests,
                         prices=vwap_dict['BTC-TUSD'],
                         signal_dict=plot_dict,
                         downsample_n=20_000,
                         ideal_bottoms=None,
                         ideal_tops=None, )

    fig.show()
    
print(f"done")

In [ ]:
one_run_params['target']

[back to top](#outline_algos_framework)

<div id='separate_model_run'> </div>

# separate model run, each part done individually

<div id='model_train_and_preds'> </div>

### model training and prediction

In [ ]:
# nb_params['single_run'] == True

In [ ]:
%%time
# del run_once_model 

if nb_params['single_run'] == True:
    try: 
        if run_once_rolling_decision_tree_model:
            raise ValueError
    except NameError:
        pass
    run_once_rolling_decision_tree_model = True

    dataset_dict = split_train_test(x, 0.98, 0.01, 0.004)  # OVERRIDE FOR SAVING SIGNAL TO RECREATE BATCHED SIGNAL RESULTS
    # dataset_dict = split_train_test(x, 
    #                                 one_run_params['targets']['p_train'], 
    #                                 one_run_params['targets']['p_test'], 
    #                                 one_run_params['targets']['p_validate'])
        
    signal_dict = rolling_decision_tree_train(prices=vwap,
                                         x_train=dataset_dict['x']['train'],
                                         x_test=dataset_dict['x']['test'],
                                         target_params=one_run_params['targets'],
                                         model_params=one_run_params['model'], ) 
    
print(f"done")

In [ ]:
if nb_params['single_run'] == True:
    # ### PLOTTING SIGNAL
    #
    plot_requests = {
        # 'start_date': (2020, 5, 10),  # ###PAUL TODO: add start / end functionality to here 
        # 'end_date': (2020, 9, 30),
        # 'p_start': 0.001,
        # 'p_end': 0.5,

        'prices': True,
        'transact_times': False,
        'preds': False,
        'smoothed_preds': True,
        'signal':True, 
        'port_val_ts': False,
        'y_train_rti': True,
        'ideal_top_bottoms': False,
    }

    fig = plot_framework(plot_requests=plot_requests,
                         prices=vwap_dict['BTC-TUSD'],
                         signal_dict=signal_dict,
                         downsample_n=20_000,
                         ideal_bottoms=None,
                         ideal_tops=None, )

    # fig.show()

In [ ]:
fig.show()

[back to top](#outline_algos_framework)

<div id='decision_algorithm'> </div>

# decision algorithm

In [ ]:
one_run_params['decision']

In [ ]:
one_run_params['decision'] = {  # TODO: implement in `algos/data/live/ports/decision_params.json
                                       'fee': 0.01,  # TODO: need to grab from port name
                                       'max_workers': 70,
                                       'cool_down': 15,
                                       'threshold': -0.09999999999999998,
                                       'pred_dist': 0.25,
                                       'price_dist': 0.0185,
                                       'stop_limit': 0.045,
                                       'overrides': ['stop_limit'],
                                       'any_two': [],
                                       'to_neutral_threshold': 0.375,
                                       'to_neutral_pred_dist': 0.14999999999999997,
                                       'to_neutral_price_dist': 0.0022500000000000003,
                                       'to_neutral_stop_limit': 0.0205,
                                       'to_neutral_overrides': ['stop_limit'],
                                       'to_neutral_any_two': []}

In [ ]:
%autoreload
from algos.decision import backtest_decision_making
nb_params= {'single_run':True}

In [ ]:
%%time
# preds = y_pred

if nb_params['single_run']: 
    framework_results = backtest_decision_making(price_series=vwap_dict['BTC-TUSD']['vwap'],
                                                 signal_dict=signal_dict,
                                                 decision_params=one_run_params['decision'])
print(f"done!")

* #### summarize results of the run 

In [ ]:
if nb_params['single_run']: 
    short_count = 0 
    long_count = 0 
    exit_short_count = 0 
    exit_long_count = 0 

    transacts_list = framework_results['transacts_list']
    
    for d in transacts_list:
        if 'short' == d['action']:
            short_count += 1
        if 'long' == d['action']:
            long_count += 1
        if 'exit_short' == d['action']:
            exit_short_count += 1
        if 'exit_long' == d['action']:
            exit_long_count += 1

    print(f"short_count: {short_count} \n"
          f"long_count: {long_count} \n"
          f"exit_short_count: {exit_short_count} \n"
          f"exit_long_count: {exit_long_count} \n")
    
    # print(f"framework_results.keys() --> {framework_results.keys()}")
    
    print(f"pnl   ----------> {framework_results['pnl']:.4f} \n"
          f"sharpe:  -------> {framework_results['sharpe']:.4f} \n"
          f"sortino:   -----> {framework_results['sortino']:.4f} \n"
          f"num_transacts: -> {framework_results['num_transacts']} \n"
         )

    # framework_results['port_value_ts'].plot()

In [ ]:
if nb_params['single_run']: 
    
    plot_requests = {
        # 'start_date': (2023, 3, 12),  # ###PAUL TODO: add start / end functionality to here 
        # 'end_date': (2023, 3, 14),
        # 'start_date': (2021, 6, 30),  # ###PAUL TODO: add start / end functionality to here 
        # 'end_date': (2022, 1, 1),
        # 'p_start': 0.9,
        # 'p_end': 0.95,
        
        'prices': True,
        'transact_times': False,
        'preds': False,
        'smoothed_preds': False,
        'signal': True,
        'port_val_ts': True,
        'y_train_rti': False,
        'ideal_top_bottoms': False,
    }

    fig = plot_framework(plot_requests=plot_requests,
                         prices=vwap_dict['BTC-TUSD']['vwap'],
                         # signal_dict = signal_dict,
                         framework_results=framework_results,
                         downsample_n=10_000,
                         ideal_bottoms=None,
                         ideal_tops=None, )

    fig.show()

In [ ]:
framework_results['port_value_ts']

In [ ]:
vwap_dict['BTC-TUSD']['vwap']

* #### DEBUG TOOL: verifying that transactions are plotted propperly 

In [ ]:
# from algos.utils import convert_date_format

# relevant_transacts = []
# for transact in transacts_list:
#     if transact['datetime'] < convert_date_format((2020, 5, 8), 'pandas'):
#         continue
#     elif transact['datetime'] > convert_date_format((2020, 5, 12), 'pandas'):
#         break
#     relevant_transacts.append(transact)
# relevant_transacts

[back to top](#outline_algos_framework) 


<div id='grid_search'> </div>

# grid search
* ### over [feature, targets, model, decision] params

In [ ]:
gs_df.sort_values(by='ranking_metric').iloc[-10:][show_cols]

# chat GPT development for hyperparemeter optimization 

In [ ]:
pip install scikit-optimize


In [ ]:
gs_df[show_cols].sort.s

In [ ]:
bayes_df = deepcopy(gs_df)

In [ ]:
params_to_opt = ['decision_cool_down',
        'decision_threshold',
        'decision_pred_dist',
        'decision_price_dist',
        'decision_stop_limit',
        'decision_to_neutral_threshold',
        'decision_to_neutral_pred_dist',
        'decision_to_neutral_price_dist',
        'decision_to_neutral_stop_limit',
        'ranking_metric', ] 


bayes_df = bayes_df[params_to_opt]

bayes_df.head()

In [ ]:
gs_df['ranking_metric'] = -1 * gs_df['ranking_metric']

In [ ]:
from skopt import gp_minimize
from skopt.space import Real, Integer

# Define the hyperparameter space
space  = [Integer(1, 60, name='decision_cool_down'),
          Real(-2.5, 2.5, name='decision_threshold'),
          Real(0.1, 1, name='decision_pred_dist'),
          Real(0.0025, 0.035, name='decision_price_dist'),
          Real(0.02, 0.05, name='decision_stop_limit'),
          Real(-1, 2, name='decision_to_neutral_threshold'),
          Real(0.05, 1, name='decision_to_neutral_pred_dist'),
          Real(0.0025, 0.03, name='decision_to_neutral_price_dist'),
          Real(0.00375, 0.0375, name='decision_to_neutral_stop_limit'),]


# Define the objective function
def objective(values):
    import pdb
    pdb.set_trace() 
    # Extract the names of the parameters from the space
    param_names = [dimension.name for dimension in space]
    params = {name: value for name, value in zip(param_names, values)}
    
    score = bayes_df.loc[(bayes_df[list(params.keys())] == pd.Series(params)).all(axis=1), 'ranking_metric'].mean()
    
    return -score  # we return the negative score because gp_minimize minimizes the function

# def objective(values):
#     params = {dim.name: value for dim, value in zip(space, values)}
#     try:
#         import
#         score = run_model_with_params(params)  # this should return the score you are trying to maximize
#         if np.isnan(score) or np.isinf(score):
#             print(f"Score is NaN or inf for params: {params}")
#             score = np.finfo(np.float64).max  # assign a large value
#     except Exception as e:
#         print(f"Exception during model run for params: {params}, exception: {str(e)}")
#         score = np.finfo(np.float64).max  # assign a large value
#     return -score  # we return the negative score because gp_minimize minimizes the function


# Run the optimizer
res = gp_minimize(objective, space, n_calls=50, random_state=0)

# The optimal parameters are in res.x
print('Best parameters:', res.x)


In [ ]:
bayes_df

In [ ]:
bayes_df.isna().sum()

# recent grid search on 5% peak signal

In [ ]:
if nb_params['gridsearch_run'] == True:
    gs_n = 4

    threshold = 0.2
    pred_dist = 0.15
    price_dist = 0.012
    stop_limit = 0.04
    # deltas
    delta_threshold = 0.9
    delta_pred_dist = 0.1
    delta_price_dist = 0.0065
    delta_stop_limit = 0.005

    exit_threshold = 0.875
    exit_pred_dist = 0.35
    exit_price_dist = 0.0085
    exit_stop_limit = 0.01625
    # deltas
    delta_exit_threshold = 0.5
    delta_exit_pred_dist = 0.2
    delta_exit_price_dist = 0.00625
    delta_exit_stop_limit = 0.00425
    

    threshold_min = threshold - delta_threshold # max(0, threshold - delta_threshold)  
    pred_dist_min = max(0, pred_dist - delta_pred_dist)
    price_dist_min = max(0, price_dist - delta_price_dist)
    stop_limit_min = max(0, stop_limit - delta_stop_limit)

    exit_threshold_min = exit_threshold - delta_exit_threshold #  max(0, exit_threshold - delta_exit_threshold)  # maybe should try letting this go negative
    # exit_threshold_min = max(0, exit_threshold - delta_exit_threshold)  # maybe should try letting this go negative
    exit_pred_dist_min = max(0, exit_pred_dist - delta_exit_pred_dist)
    exit_price_dist_min = max(0, exit_price_dist - delta_exit_price_dist)
    exit_stop_limit_min = max(0, exit_stop_limit - delta_exit_stop_limit)


    threshold_max = threshold + delta_threshold
    pred_dist_max = pred_dist + delta_pred_dist
    price_dist_max = price_dist + delta_price_dist
    stop_limit_max = stop_limit + delta_stop_limit
    exit_threshold_max = exit_threshold + delta_exit_threshold
    exit_pred_dist_max = exit_pred_dist + delta_exit_pred_dist
    exit_price_dist_max = exit_price_dist + delta_exit_price_dist
    exit_stop_limit_max = exit_stop_limit + delta_exit_stop_limit

    decision_dict = \
        {
            'fee': [0.01],  # FEE IN PERCENT. 
            # 'preds_out_ewm': [0.45, 0.575],  # ###PAUL this goes to model
            'max_workers': [80],
            'cool_down': [10, 45],

            'threshold': list(np.linspace(start=threshold_min, stop=threshold_max, num=gs_n)),
            'pred_dist': list(np.linspace(start=pred_dist_min, stop=pred_dist_max, num=gs_n)),
            'price_dist': list(np.linspace(start=price_dist_min, stop=price_dist_max, num=gs_n-1)),
            'stop_limit': list(np.linspace(start=stop_limit_min, stop=stop_limit_max, num=gs_n-1)),
            # 'stop_limit': [0.0088, 0.01, 0.015, ],
            'overrides': [['stop_limit'], ],
            'any_two': [[]], # , ['threshold', 'pred_dist', 'price_dist']],

            'to_neutral_threshold': list(np.linspace(start=exit_threshold_min, stop=exit_threshold_max, num=gs_n)),
            'to_neutral_pred_dist': list(np.linspace(start=exit_pred_dist_min, stop=exit_pred_dist_max, num=gs_n)),
            'to_neutral_price_dist': list(np.linspace(start=exit_price_dist_min, stop=exit_price_dist_max, num=gs_n-1)),
            'to_neutral_stop_limit': list(np.linspace(start=exit_stop_limit_min, stop=exit_stop_limit_max, num=gs_n-1)),
            # 'to_neutral_stop_limit': [0.005, 0.009, 0.0125, ],
            'to_neutral_overrides': [['stop_limit'], ],
            'to_neutral_any_two': [[]], # , ['threshold', 'pred_dist', 'price_dist'], ],  or ['pred_dist', 'price_dist']
        }

print(f"done")

In [ ]:
%%time 
if nb_params['gridsearch_run'] == True:
    try:
        if run_once_grid_search:
            raise ValueError
    except NameError:
        run_once_grid_search = True
        

    gridsearch_params = \
        {
            'features':
                {
                    # ###PAUL TODO: 
                }, 
            'targets':
                {
                    'p_train': [0.01],
                    'p_test': [0.01], # temp to speed things up [0.3],
                    'p_validate': [0.001],
                    'min_peak_dist': [25],
                    'peak_prominence': [0.04, ],  # 0.06, 0.08],
                    'interval_len_in_steps': [14400],
                    'smoothing_window': [None],
                    'pd_ewm_alpha': [0.025],
                    'training_days': [630],  # ###PAUL training days should be removed model, all rely on targets dict 
                },
            'model':
                {
                    # light gbm params # ###PAUL eventuall move seprate out model params to generalizem more
                    "model": [None],
                    "learning_rate": [0.01],
                    "max_depth": [10],
                    "num_leaves": [6],
                    "n_estimators": [88],
                    "verbose": [-1],  # silences the model built in

                    # backtest_rolling params
                    'training_days': [630],
                    'f_window': [0],
                    'step_size': [5],
                    'preds_pd_ewm_alpha': [0.025],
                    'rolling_normalize_preds': [True],
                    'signal_norm_window': [5760],
                },

            'decision': decision_dict
        }

    print(f"STARTING GRIDSEARCH")
    # results_df = grid_search_over_parameters(x, prices['vwap'], gridsearch_params, )

print(f"done")

* ### grid search over `signal_dict`

In [ ]:
from algos.gridsearch import make_list_of_combo_dicts, decision_multiprocess

target_params_nn_temp = {'not_ready_for_nn': 'skip'}
model_params_nn_temp = {'not_ready_for_nn': 'skip'}
decision_combos = make_list_of_combo_dicts(gridsearch_params['decision'])
                     
gs_df = decision_multiprocess(decision_combos=decision_combos,
                                signal_dict=signal_dict,
                                prices=vwap,
                                target_params=target_params_nn_temp,
                                model_params=model_params_nn_temp,
                                max_workers=80)

gs_df['ranking_metric'] = gs_df['sharpe'] * gs_df['pnl'] / gs_df['num_transacts' ] 

print(f"done") 

<div id='one_model_run'> </div>

# one model run
* #### check `one_run_params`

In [ ]:
if nb_params['one_full_run'] == True:
    try:
        if run_once_single_model:
            raise ValueError
    except NameError:
        run_once_single_model = True

    framework_results = run_full_process(x, prices['vwap'], run_on_validation_period=False, **one_run_params)
    drawdowns, recoveries = analyze_framework_run(framework_results=framework_results, ts_freq='min')
print('done')

In [ ]:
if nb_params['one_full_run'] == True:
    plot_requests = {
        # 'start_date': (2020, 4, 20),  # ###PAUL TODO: add start / end functionality to here 
        # 'end_date': (2020, 4, 23),
        # 'p_start': 0.0,
        # 'p_end': 3/7,

        'prices': True,
        'transact_times': True,
        'preds': False,
        'smoothed_preds': True,
        'port_val_ts': True,
        'y_train_rti': False,
        'ideal_top_bottoms': False,
    }

    fig = plot_framework(plot_requests=plot_requests,
                         prices=prices['vwap'],
                         framework_results=framework_results,
                         downsample_n=15_000,
                         ideal_bottoms=None,
                         ideal_tops=None, )

    fig.show()

[back_to_top](#outline_algos_framework) 


<div id='neural_net_dev'> </div> 

* # neural_net_dev
    * [mlp - batched](#mlp_batched)
    * [mlp - rolling](#mlp_rolling)

<div id='mlp_batched'> </div> 

## batched mlp 


In [ ]:
# del run_once_norm_whole_dataset

In [ ]:
%%time 

if nb_params['MLP_batch']: 
    try:
        if run_once_norm_whole_dataset:
            raise ValueError
    except NameError:
        run_once_norm_whole_dataset = True

    # normalize the feature set in a rolling manner.. 
    rolling_normalization_n_obvs = 30*24*60
    x_normed_rti = ((x - x.rolling(rolling_normalization_n_obvs).mean()).fillna(0) /
                    x.rolling(rolling_normalization_n_obvs).std().fillna(method='ffill')).iloc[rolling_normalization_n_obvs-1:]
    y_normed_rti = ((y - y.rolling(rolling_normalization_n_obvs).mean()) /
                    y.rolling(rolling_normalization_n_obvs).std().fillna(method='ffill')).iloc[rolling_normalization_n_obvs-1:]

    # get rid of columns without enough data 
    nan_count_by_col = x_normed_rti.isna().sum()
    over_thresh = 0.05 < nan_count_by_col/x_normed_rti.shape[0] # identify all columns with more than 5% missing 
    x_normed_rti = x_normed_rti[over_thresh.index[~over_thresh]]

    x_normed_rti = x_normed_rti.fillna(method='ffill').fillna(method='bfill').fillna(value=0)
    y_normed_rti = y_normed_rti.fillna(method='ffill').fillna(method='bfill').fillna(value=0)

    x_normed_rti = np.clip(a=x_normed_rti, a_min=-10, a_max=10)
    y_normed_rti = np.clip(a=y_normed_rti, a_min=-10, a_max=10)

    x_normed_rti, y_normed_rti = cut_two_pd_dti_objects_to_matching_dtis(x_normed_rti, y_normed_rti)

    print(f"CHECKING ARRAYS (sometimes weird infs / -infs  pop up)")
    print(f"min(x_normed_rti) ---- {np.min(x_normed_rti.values)}")
    print(f"max(x_normed_rti) ---- {np.max(x_normed_rti.values)}")
    print(f"mean(x_normed_rti) ---- {(x_normed_rti.values).mean()}")
    print(f"std(x_normed_rti) ---- {(x_normed_rti.values).std()}")
    print(f"min(y_normed_rti) ---- {min(y_normed_rti)}")
    print(f"max(y_normed_rti) ---- {max(y_normed_rti)}")
    print(f"mean(y_normed_rti) ---- {(y_normed_rti.values).mean()}")
    print(f"std(y_normed_rti) ---- {(y_normed_rti.values).std()}")

print(f"done")

* #### create batched model for training 
    * split in this cell because cleaning above will affect model via `feature_in_dim`

In [ ]:
# override_model = True

In [ ]:
%%time 

if nb_params['MLP_batch']:
    # dataset_dict = split_train_test(x_normed_rti, 0.50, 0.15, 0.01)    # <<---- TEST PERIOD
    # dataset_dict = split_train_test(x_normed_rti, 0.65, 0.15, 0.005)    # <<---- TEST PERIOD
    dataset_dict = split_train_test(x_normed_rti, 0.55, 0.35, 0.005)    # <<---- TEST PERIOD ---- THIS IS A TEMP NUMBER TO GET A SIGNAL FOR 2021 FOR YOHAN     
    # dataset_dict = split_train_test(x_normed_rti, 0.8, 0.195, 0.005)    # <<---- TEST PERIOD ---- THIS IS A TEMP NUMBER TO GET A SIGNAL FOR 2022 FOR YOHAN 
    # dataset_dict = split_train_test(x_normed_rti, 0.8, 0.15, 0.01)   # <<---- VALIDATE PERIOD
    feature_in_dim = dataset_dict['x']['train'].shape[1]
    
    # make dataloaders for batched training 
    train_dataloader = make_mlp_dataloader(x_in=dataset_dict['x']['train'], batch_size=24*60, y_in=y_normed_rti, shuffle=False) 
    test_dataloader = make_mlp_dataloader(dataset_dict['x']['test'], batch_size=24*60, y_in=y_normed_rti, shuffle=False) 
    test_dataloader_no_targets = make_mlp_dataloader(dataset_dict['x']['test'], batch_size=24*60, y_in=None, shuffle=False)
    
    # dataset_dict = split_train_test(x_normed_rti, 0.65, 0.15, 0.15)   
    # dataset_dict = split_train_test(x_normed_rti, 0.8, 0.075, 0.075)   
    # train_dataloader = make_mlp_dataloader(x_in=dataset_dict['x']['test'], batch_size=24*60, y_in=y_normed_rti, shuffle=False) 
    # test_dataloader = make_mlp_dataloader(dataset_dict['x']['validate'], batch_size=24*60, y_in=y_normed_rti, shuffle=False) 
    # test_dataloader_no_targets = make_mlp_dataloader(dataset_dict['x']['validate'], batch_size=24*60, y_in=None, shuffle=False)

    # Get cpu or gpu device for training.
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print('using ---> ' + device)

    # ### adjustable parameters
    #
    #
    # loss functions 
    loss_fn = nn.MSELoss()
    # loss_fn = nn.L1Loss()
    
    try:
        if override_model == True:
            print(f"overrided model weights")
            model = NeuralNetwork(feature_in_dim=feature_in_dim).to(device)
        else:
            print(f"model weights left unchanged") 
    except NameError:
        try:
            model
            print(f"model weights left unchanged") 
        except NameError:
            print(f"model did not exist, so made one")
            model = NeuralNetwork(feature_in_dim=feature_in_dim).to(device)
    override_model = False
        
    # optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    torch.set_num_threads(80)

    print(model)
    
print(f"done!")

* #### run batched training 

In [ ]:
run_once_mlp_batch_train = True 
del run_once_mlp_batch_train
    
if nb_params['MLP_batch']:
    try: 
        if run_once_mlp_batch_train == True:
            raise ValueError
    except NameError:
        pass
    run_once_mlp_batch_train = True

    # ## set number of epochs
    #
    epochs = 10
    
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        torch.set_num_threads(80)
        train(train_dataloader, model, loss_fn, optimizer, printout_level='Low')
        torch.set_num_threads(80)
        test_loss = test(test_dataloader, model, loss_fn)
        print(f"-- test_loss: {test_loss}") 
        
print("done!")

* #### create predictions for batch of observations

In [ ]:
if nb_params['MLP_batch']:
    
    # ###PAUL TODO: handle this better ---- used between batch and rolling
    model_params = \
        {
            # ROLLING train / test functionality only 
            'train_days': 77,
            'step_size': 14,
            'f_window': 0,
            'num_epochs_for_train': 20,
            'num_epochs_when_rolling': 4,
            'batch_size': 24 * 60, 

            # ###PAUL TODO: build the target post processing into `make_targets()` adding  `target_params` as input
            'volatility_normalization_for_targets': False,
            'return_window_n': 2*60,         # REQUIRED IF `volatility_normalization_for_targets` == True
            'variance_window_n': 2*24 * 60,  # REQUIRED IF `volatility_normalization_for_targets` == True
        
            # make target series used for training have standard normal distribution 
            'normalize_targets': True, 

            # 'optimizer': torch.optim.SGD(model.parameters(), lr=1e-4),
            'optimizer': torch.optim.Adam(model.parameters(), lr=1e-3),
            # 'loss_fn': nn.L1Loss(),
            'loss_fn': nn.MSELoss(),
        
            # dataloader shuffle stuff (planned for batch training, but may want to add to rolling) 
            'shuffle_train': False,
            'shuffle_test': False,  # this should always be False, maybe dont give it an argument

            # ### preds post processing parameters to make preds into a signal 
            'rolling_normalize_preds': True,
            'preds_pd_ewm_alpha': 0.01,
            'signal_norm_window': 5*24*60,   # REQUIRED IF `rolling_normalize_preds` == True
        }
    
    # preds = make_predictions_for_dataloader_features_only(test_dataloader_no_targets, model)
    # preds = reshape_preds(preds)
    # preds = pd.Series(preds, index=dataset_dict['x']['test'].index)
    # # preds = pd.Series(preds, index=dataset_dict['x']['validate'].index)
        
    signal_dict = make_signal_dict_from_preds(
                                              # preds, 
                                              signal_dict['preds'],
                                              y_train_rti=y_normed_rti, 
                                              target_params=one_run_params['targets'],
                                              model_params=model_params,
                                              model=model, 
                                              # model=signal_dict['model'], 
                                             )

print(f"done!")

[back to top](#outline_algos_framework) 

<div id='mlp_rolling'> </div> 

* # rolling MLP train / test 

In [ ]:
del run_once_norm_whole_dataset

In [ ]:
x.shape

In [ ]:
%%time

if nb_params['MLP_rolling'] == True:
    try:
        if run_once_norm_whole_dataset:
            raise ValueError
    except NameError:
        run_once_norm_whole_dataset = True

    # normalize the feature set in a rolling manner.. 
    preprocess_rolling_norm_n_obvs = one_run_params['feature']['preprocess_rolling_norm_n_obvs']
    x_normed_rti = ((x - x.rolling(preprocess_rolling_norm_n_obvs).mean()).fillna(0) /
                    x.rolling(preprocess_rolling_norm_n_obvs).std().fillna(method='ffill')).iloc[
                   preprocess_rolling_norm_n_obvs - 1:]

    # get rid of columns without enough data 
    nan_count_by_col = x_normed_rti.isna().sum()
    over_thresh = 0.05 < nan_count_by_col / x_normed_rti.shape[0]  # identify all columns with more than 5% missing 
    x_normed_rti = x_normed_rti[over_thresh.index[~over_thresh]]

    x_normed_rti = x_normed_rti.fillna(method='ffill').fillna(method='bfill').fillna(value=0)
    x_normed_rti = np.clip(a=x_normed_rti, a_min=-10, a_max=10)

    feature_in_dim = x_normed_rti.shape[1]

    # og_x_normed_rti = deepcopy(x_normed_rti)  # ###PAUL_del_later make a copy of x_normed for now 

    print(f"done")

# # ### saving preprocessed features in cache (incase something happens) 
# x_normed_rti = deepcopy(og_x_normed_rti)  ### bring back the original x normed easily 

In [ ]:
check_prices_for_gaps(x_normed_rti)

In [ ]:
x_normed_rti.shape

In [ ]:
x_normed_rti.columns

In [ ]:
# rolling_normed_feature_set_fp =  f"{params['dirs']['data_dir']}features/rolling_normed_30_d.pickle"
# # x_normed_rti.to_pickle(rolling_normed_feature_set_fp)
# x_normed_rti = pd.read_pickle(rolling_normed_feature_set_fp)
# vwap_prices_fp = f"{params['dirs']['data_dir']}prices/just_vwap.pickle"
# # prices['vwap'].to_pickle(vwap_prices_fp)
# vwap_prices = pd.read_pickle(vwap_prices_fp)

* ### run rolling training process 

In [ ]:
nb_params['MLP_rolling'] = True
del run_once_mlp_rolling_train

In [ ]:
x_normed_rti

In [ ]:
%%time 


run_once_mlp_rolling_train = False
if nb_params['MLP_rolling'] == True:
    
    try:
        if run_once_mlp_rolling_train:
            print(f"`run_once_mlp_rolling_train` must be deleted or set False")
            raise ValueError
        run_once_mlp_rolling_train = True
    except NameError: 
        run_once_mlp_rolling_train = True
        pass
    
    # ### data splitting
    #
    sd = convert_date_format((2018, 1, 1), 'pandas')  
    ed = convert_date_format((2023, 7, 18), 'pandas')   # FOR TEST 
    # ed = convert_date_format((2023, 5, 19), 'pandas')   # FOR VALIDATION
    x_for_roll = x_normed_rti[np.logical_and(sd < x_normed_rti.index, x_normed_rti.index < ed) ]  
    
    
    feature_in_dim = x_for_roll.shape[1]                  
      
    # Get cpu or gpu device for training.
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print('using ---> ' + device)

    # ### adjustable parameters
    #
    # loss functions 
    loss_fn = nn.MSELoss()
    # loss_fn = nn.L1Loss()

    model = NeuralNetwork(feature_in_dim=feature_in_dim).to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
    # optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    
    print(model)

    target_params= \
        {
         'min_peak_dist': 480,
         'peak_prominence': 0.055,
         'interval_len_in_steps': 35*24*60,
         'pd_ewm_alpha': 0.01
    }
    

    model_params = \
        {
            'train_days': 365,
            'step_size': 3,
            'f_window': 0,

            'num_epochs_for_train': 25,
            'num_epochs_when_rolling': 5,
            # 'num_epochs_for_train': 2,
            # 'num_epochs_when_rolling': 1,
                
            'batch_size': 24 * 30,
            
            # dataloader shuffle stuff (planned for batch training, but may want to add to rolling) 
            'shuffle_train': False,
            'shuffle_test': False,  # this should always be False, maybe dont give it an argument
        
            'volatility_normalization_for_targets': False,  # ###PAUL TODO: check this calculation, OFF for now
            'return_window_n': 3*60,         # REQUIRED IF `volatility_normalization_for_targets` == True
            'variance_window_n': 3*24*60,    # REQUIRED IF `volatility_normalization_for_targets` == True

            'normalize_targets': True,  # normalizes targets per each iteration BEFORE cutting them for training

            'optimizer': optimizer,
            # 'loss_fn': nn.L1Loss(),
            'loss_fn': nn.MSELoss(),
       
            # ### preds post processing parameters to make preds into a signal 
            'preds_pd_ewm_alpha': 0.01,
            'rolling_normalize_preds': True,
            'signal_norm_window': 5*24*60,   # REQUIRED IF `rolling_normalize_preds` == True
           
            # 'lstm_len': 120,  # ###PAUL this wil need to be worked into the index`
            # # # needs to be subtracted from the start of the of the training data but more importantly the start of
            # # the test data... implications if f_window==0...? technically the first `lstm_len` of obvservations
            # # of the test period would contain info from the training period, though this would match real live
            # # at midnight right after training, of course if f_window >= 1 then we have no problem
            # # ... but this initially came into my head on how to create a continuous output series for plotting...
            # # soo, to handle that - subtract `lstm_len` from start_test_ioc, should handle it.
            # # also has affects on the train end because I want to continue to support outputting the training targets
        }

    print(f"got here")
    signal_dict = rolling_train_and_backtest_mlp(  x=x_for_roll,
                                                   prices=vwap_dict['BTC-TUSD']['vwap'],
                                                   model=model,
                                                   feature_params=feature_params, 
                                                   target_params=target_params,
                                                   model_params=model_params,
                                                   printout_level='Low')

print(f"done")


[back to top](#outline_algos_framework)

* ### signal analysis and threshold / time hold based decision framework_results
* #### make `signal_dict` for batched method
    * using the model to predict a signal, then some postprocessing on predictions 

In [ ]:
# single_pair_signal_analysis = analyze_signal(vwap_dict['BTC-TUSD'], single_pair_signal_dict, signal_series_name='signal', target_series_name='y_train_rti')
# print_signal_analysis(single_pair_signal_analysis)

In [ ]:
signal_analysis = analyze_signal(vwap_dict['BTC-TUSD'], signal_dict, signal_series_name='signal', target_series_name='y_train_rti')
print_signal_analysis(signal_analysis)

In [ ]:
signal_analysis = analyze_signal(vwap_dict['BTC-TUSD'], signal_dict, signal_series_name='signal', target_series_name='y_train_rti')
print_signal_analysis(signal_analysis)

In [ ]:
neg_signal_dict = deepcopy(signal_dict) 
neg_signal_dict['signal'] = -1 * neg_signal_dict['signal']

In [ ]:
%autoreload
from algos.decision import run_simple_signal_threshold_and_hold_time_based_framework

In [ ]:
%%time 

framework_results = \
    run_simple_signal_threshold_and_hold_time_based_framework(
                                                              prices=vwap_dict['BTC-TUSD'],
                                                              signal_dict=signal_dict,
                                                              thresh=2,
                                                              hours=5*24,
                                                              trigger_on='back_under',
                                                              # trigger_on='break_above',
                                                              fee_in_percent=0,    )

In [ ]:
%%time 

framework_results = \
    run_simple_signal_threshold_and_hold_time_based_framework(
                                                              prices=vwap_dict['BTC-TUSD'],
                                                              signal_dict=neg_signal_dict,
                                                              thresh=2,
                                                              hours=20,
                                                              # trigger_on='back_under',
                                                              trigger_on='break_above',
                                                              fee_in_percent=0,      
                                                              )

In [ ]:
neg_signal_dict.keys()

In [ ]:
neg_signal_dict['signal']

In [ ]:
neg_signal_dict['signal'][neg_signal_dict['signal'].index.year == 2022]

In [ ]:

# # ### NAME OF `signal_dict` run
# #
# #
# signal_dict_name = f"signal_dict____2023_08_01____mlp_rolling____first_multi_asset_feature_set"  
# signal_dict_name = f"signal_dict____2023_08_03___mlp_rolling____validate"  
# signal_dict_name = f"signal_dict____2023_08_23___mlp_rolling____to_2023_07_18"
signal_dict_name = f"signal_dict____2023_09_01___mlp_rolling_smaller_model"
# signal_dict_name = f"signal_dict____2023_09_01___mlp_rolling____large_model"
# signal_dict_name = f"signal_dict____2023_09_01___mlp_rolling____large_model_bigger_learning_rate"
# signal_dict_name = f"signal_dict____2023_09_01___mlp_rolling____large_model_SGD_1e4"

signal_dict_fp = f"{params['dirs']['data_dir']}pickled_signal_dicts/{signal_dict_name}.pickle"

# # ### SAVING
# #
# if os.path.isfile(signal_dict_fp): 
#     print(f"signal already exists at that filepath") 
# else:
#     pickle.dump(signal_dict,  #   <<<------- COPY PASTE DESIRED VARIABLE HERE
#                 open(signal_dict_fp, "wb"))


# # ### OPENING
# #
with open(signal_dict_fp, 'rb') as f:
     signal_dict = pickle.load(f) #   <<<------- COPY PASTE DESIRED VARIABLE HERE

### LEAVE ON ALWAYS
#
del signal_dict_name; del signal_dict_fp

In [ ]:
neg_signal_dict = deepcopy(signal_dict) 
neg_signal_dict['signal'] = -1 * neg_signal_dict['signal']

In [ ]:
signal_dict['model']

In [ ]:
signal_dict['model']

# results of smaller model below 

In [ ]:
# if nb_params['neural_net_dev'] == True:
    
#     # ### PLOTTING SIGNAL
#     #
#     plot_requests = {
#         # 'start_date': (2023, 2, 1),  # ###PAUL TODO: add start / end functionality to here 
#         # 'end_date': (2023, 7, 1),
#         # 'p_start': 0.5,
#         # 'p_end': 0.69,

#         'prices': True,
#         'transact_times': False,
#         'preds': False,
#         'smoothed_preds': True,
#         'signal': True,
#         'port_val_ts': True,
#         'y_train_rti': False,
#         'ideal_top_bottoms': False,
#     }

#     fig = plot_framework(plot_requests=plot_requests,
#                          prices=vwap_dict['BTC-TUSD']['vwap'],
#                          framework_results=framework_results, 
#                          downsample_n=20_000, 
#                          ideal_bottoms=None, 
#                          ideal_tops=None, )

#     fig.show()

In [ ]:
years = [2019, 2020, 2021, 2022, 2023]
port_vals_to_concat = [] 

temp_signal_dict = deepcopy(neg_signal_dict) 


framework_results = \
    run_simple_signal_threshold_and_hold_time_based_framework(
                                                              prices=vwap_dict['BTC-TUSD'],
                                                              signal_dict=neg_signal_dict,
                                                              thresh=2,
                                                              hours=20,
                                                              # trigger_on='back_under',
                                                              trigger_on='break_above',
                                                              fee_in_percent=0,      
                                                              )
for year in years:
    years_signal = neg_signal_dict['signal'][neg_signal_dict['signal'].index.year == year]
    temp_signal_dict['signal'] = years_signal 
    
    print(f"-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=\n"
          f"RUNNING BACKTEST FOR {year} \n")
          
    framework_results_t = \
        run_simple_signal_threshold_and_hold_time_based_framework(
                                                                  prices=vwap_dict['BTC-TUSD'],
                                                                  signal_dict=temp_signal_dict,
                                                                  thresh=2,
                                                                  hours=20,
                                                                  # trigger_on='back_under',
                                                                  trigger_on='break_above',
                                                                  fee_in_percent=0,      
                                                                  )
    
    port_vals_to_concat.append(framework_results_t['port_value_ts']) 
    
    
framework_results['port_value_ts'] = pd.concat(port_vals_to_concat) 

In [ ]:
if nb_params['neural_net_dev'] == True:
    
    # ### PLOTTING SIGNAL
    #
    plot_requests = {
        # 'start_date': (2023, 2, 1),  # ###PAUL TODO: add start / end functionality to here 
        # 'end_date': (2023, 7, 1),
        # 'p_start': 0.5,
        # 'p_end': 0.69,

        'prices': True,
        'transact_times': False,
        'preds': False,
        'smoothed_preds': True,
        'signal': True,
        'port_val_ts': True,
        'y_train_rti': True,
        'ideal_top_bottoms': False,
    }

    fig = plot_framework(plot_requests=plot_requests,
                         prices=vwap_dict['BTC-TUSD']['vwap'],
                         framework_results=framework_results, 
                         downsample_n=20_000, 
                         ideal_bottoms=None, 
                         ideal_tops=None, )

    # fig.show()
    
years = [2019, 2020, 2021, 2022, 2023]
for year in years:
    fig.add_shape(
        go.layout.Shape(
            type="line",
            x0=f"{year}-01-01",
            x1=f"{year}-01-01",
            y0=0,
            y1=60000,
            # xref="x",
            yref="paper",
            line=dict(color="red", width=1.5), 
            # row=1, 
            # col=1
        ),
        row=1, col=1
    )

fig.show()

In [ ]:
2+2**9*(11.07**(1/2))

In [ ]:


# Function to process and write trades to ClickHouse
def process_trade_data(trade):
    # Connect to ClickHouse
    client = Client('localhost')

    # Example schema for your trade data
    trade_data = {
        'timestamp': trade['timestamp'],
        'symbol': trade['symbol'],
        'price': trade['price'],
        'amount': trade['amount']
    }

    # Insert into ClickHouse (modify table name and columns as per your schema)
    client.execute('INSERT INTO trades (timestamp, symbol, price, amount) VALUES', [trade_data])

# Initialize


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np

# Create a sample time series dataframe
date_rng = pd.date_range(start='2019-01-01', end='2023-12-31', freq='D')
plot_port_vals = pd.Series(np.random.randn(len(date_rng)), index=date_rng)
plot_port_vals2 = pd.Series(np.random.randn(len(date_rng)), index=date_rng)

# Create subplots
fig = make_subplots(rows=2, cols=1)

# Add scatter plots
fig.add_trace(go.Scattergl(x=plot_port_vals.index, y=plot_port_vals, name='port_val'), row=1, col=1)
fig.add_trace(go.Scattergl(x=plot_port_vals2.index, y=plot_port_vals2, name='port_val_2'), row=2, col=1)

# Add vertical lines at the start of each year
years = [2019, 2020, 2021, 2022, 2023]
for year in years:
    fig.add_shape(
        go.layout.Shape(
            type="line",
            x0=f"{year}-01-01",
            x1=f"{year}-01-01",
            y0=0,
            y1=1,
            xref="x",
            yref="paper",
            line=dict(color="red", width=1.5),
        ),
        row=1,
        col=1
    )

fig.show()


* #### find strategy analysis function and put here 


[back to top](#outline_algos_framework) 

<div id='neural_net_lstm'> </div> 

## lstm

##### reshaping 
* #### $X: \mathbb{R}^{n, p} \rightarrow  \mathbb{R}^{n, p} $ 

* #### $1: \mathbb{R}^{n, 1} \rightarrow  \mathbb{R}^{n-s, 1}$ 

resources for transition to LSTM: 
* [pytorch beginner lstm](https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html)
* examples
    * [How to use PyTorch LSTMs for time series regression](https://www.crosstab.io/articles/time-series-pytorch-lstm/#evaluation)
    * [LSTM for Time Series Prediction in PyTorch](https://machinelearningmastery.com/lstm-for-time-series-prediction-in-pytorch/)
    * [Time Series Forecasting with Deep Learning in PyTorch (LSTM-RNN)](https://towardsdatascience.com/time-series-forecasting-with-deep-learning-in-pytorch-lstm-rnn-1ba339885f0c) 
* multi-timescale inspiration 
    * [Multi-timescale Representation Learning in LSTM Language Models](https://openreview.net/forum?id=9ITXiTrAoT)
    * 

[back to top](#outline_algos_framework) 


<div id='production_instruction'> </div>div> 


# **production instruction** notes on how things are currently ran 

* outline
    * [clickhouse ---- algos_db](#clickhouse_algos_db)  

<div id='clickhouse_algos_db'> </div>div> 

## clickhouse ---- algos_db

* setup commands (of course via chat gpt so maybe check some specific values if setting up as a script
    * ```sudo apt update```
    * ```sudo apt upgrade```
    * ```sudo apt-key adv --keyserver keyserver.ubuntu.com --recv E0C56BD4```
    * ```echo "deb http://repo.yandex.ru/clickhouse/deb/stable/ main/" | sudo tee /etc/apt/sources.list.d/clickhouse.list```
    * ```sudo apt update```
    * ```sudo apt install -y clickhouse-server clickhouse-client```
    * ```sudo systemctl start clickhouse-server```
    * ```sudo systemctl enable clickhouse-server```
    * 



[back to top](#outline_algos_framework) 

<div id='live_functionality'> </div> 

* # short term **current** development and live functionality 
* outline 
    * [live_framework_adjustments](#live_framework_adjustments)  
    * [async runner example](#async_runner_example) 
    * [EAORS integration](#EAORS_integration)
    * [start up configurations](#start_up_configs)
    * [`state_dict` split to `state` and `data` dicts](#state_dict_splitting) 
    * [liquidation map](#liquidation_map) 
    * [price volume profile](#price_volume_profile)
    * [dash app](#dash_app)
    * [continious live development scratch space](#live_dev_scratch_space) 


<div id='live_framework_adjustments'> </div> 

## live_framework_adjustments

In [ ]:
!pip install ipdb

<div id='async_runner_example'> </div> 

### async runner example

In [ ]:
if nb_params['live_trade_work'] == True:

    import asyncio
    import os
    import time


    class Runner:
        def __init__(
            self,
            config,
            logger,
        ):
            self.config = config
            self.logger = logger
            self.event_loop = asyncio.get_event_loop()

        def run(self):
            self._print_diagnostics()
            failure_times = []
            sleep_time = 10
            while True:
                try:
                    asyncio.run(self._looper())
                except:
                    self.logger.exception("Encountered exception. Restarting")
                failure_times.append(time.time())
                if len(failure_times) > 5 and (time.time() - failure_times[-5]) < (60 * 20):
                    self.logger.error("Failed 5 times in 20 minutes: Aborting.")
                    break
                self._cancel_tasks()
                self.logger.info(f"Sleeping for {sleep_time}")
                time.sleep(sleep_time)

        def _print_diagnostics(self):
            version_file = os.path.expanduser("~/VERSION.txt")
            if os.path.exists(version_file):
                with open(version_file, "r") as f:
                    version = f.read()
                self.logger.info(f"algos_dbs_dbs_dbs_dbs_dbs version={version.strip()}")
            else:
                self.logger.info(f"Unable to determine algos_dbs_dbs_dbs_dbs_dbs version information")
            self.logger.info(f"Config file: {self.config}")

        def _cancel_tasks(self):
            # TODO: Fairly sure this doesn't do anything since asyncio.run creates a new task each time
            #     self.event_loop is the loop for the first run, but every subsequent run will be different
            self.logger.warning("Killing all the tasks!")
            for task in asyncio.all_tasks(self.event_loop):
                task.cancel()

        async def _looper(self):
            raise NotImplementedError("Child class must implement _looper function!")


* #### applying this to the algos framework

In [ ]:
if nb_params['live_trade_work'] == True:
    from algos.runner import Runner
    from src.util import get_logger, get_secret


    class AlgosFrameworkRunner(Runner):
        async def _looper(self):
            loop = asyncio.get_event_loop()
            update_queue = asyncio.Queue(loop=loop)
            order_queue = asyncio.Queue(loop=loop)

            instruments = ['BTC-USDT']

            # TODO: Dynamically load the exchanges
            binance = Binance(
                instrument=instruments,
                api_key=get_secret("BINANCE_API_KEY_1"),
                api_secret=get_secret("BINANCE_SECRET_KEY_1"),
            )


    # def get_args():
    #     parser = argparse.ArgumentParser()
    #     parser.add_argument(
    #         "-c",
    #         "--config-dir",
    #         help="Path to directory to find config files",
    #         required=True,
    #     )
    #     parser.add_argument(
    #         "-i", "--instrument", default="all", help="Instruments to run for"
    #     )
    #     parser.add_argument(
    #         "-s", "--sleep", action="store_true", help="Sleep for a random amount of time"
    #     )
    #     return parser.parse_args()


    # def main():


    # args = get_args()
    configs = []

    logger = get_logger(f"algos-framework")

    if args.sleep:
        sleep_time = int(30 * random.random())
        logger.info(f"Sleeping for {sleep_time} seconds before launching")
        time.sleep(sleep_time)
    runner = AlgosFrameworkRunner(configs, logger)
    runner.run()


    # if __name__ == "__main__":
    #     main()


<div id='EAORS_integration'> </div> 

### EAORS integration

In [ ]:
if nb_params['live_trade_work'] == True:
    instruments = ['BTC-BUSD']
    
    # TODO: Dynamically load the exchanges
    binance = Binance(
        instrument=instruments,
        api_key=get_secret("BINANCE_API_KEY_1"),
        api_secret=get_secret("BINANCE_SECRET_KEY_1"),
    )

    task = asyncio.create_task(binance.start())
    
    # error line # comment out this line to run the cell
    res_place_order = await binance.place_order(instrument='BTC-BUSD',
                                                size=0.0006295907660020986,
                                                order_type=OrderType.LIMIT_ORDER,
                                                direction=OrderDirection.BUY,
                                                price = 23801.99,
                                                time_in_force = None,
                                                 )
    
print(f"done")

* #### exploring EAORS responses

In [ ]:
# binance.order_queue

In [ ]:
# res_place_order

In [ ]:
# error line # comment out this line to run the cell

In [ ]:
if nb_params['live_trade_work'] == True:
    res_place_order_sell = await binance.place_order(instrument='BTC-BUSD',
                                                size=0.0006295907660020986,
                                                order_type=OrderType.LIMIT_ORDER,
                                                direction=OrderDirection.SELL,
                                                price = 23800.05,
                                                time_in_force = None,
                                                )

In [ ]:
# res_place_order_sell

In [ ]:
# binance.order_queue

In [ ]:
if nb_params['live_trade_work'] == True:
    res_get_balance = binance.get_balance()
    res_get_balance

In [ ]:
if nb_params['live_trade_work'] == True:
    res_get_orders = await binance.get_orders()
    res_get_orders

[back to top](#outline_algos_framework) 

<div id='state_dict_splitting'> </div> 


* ### `state_dict` split to `state` and `data` dicts
    * finish [signal_revamp](#signal_revamp) above first, that way wer can get a fresh state dict going 

In [ ]:
# feature_params = signal_dict['feature_params']
# model_params = signal_dict['model_params']
# model = signal_dict['model']

In [ ]:
state_dict_fp = f"{params['dirs']['data_dir']}live/ports/prod_1____BTC_USDT_trades_only_data/state_dict.pickle"

# OPENING
with open(state_dict_fp, 'rb') as handle:
     state_dict  = pickle.load(handle) #   <<<------- COPY PASTE DESIRED VARIABLE HERE

In [ ]:
state_dict.keys()

In [ ]:
state_dict['LS']

In [ ]:
oos = ccxt_client.fetch_open_orders('BTCTUSD')

[back to top](#outline_algos_framework) 

<div id='liquidation_map'> </div> 

* ### liquidation map

In [ ]:
def load_liquidation_map(decay, window_decay, path = '/opt/shared/pchen/liquidation_maps/data/bitfinex_distrib_long/'):
    
    #l_maps = pd.read_csv(f"/opt/shared/pchen//liquidation_maps/data/hourly_decay_{decay}_rol_wind_size_720_v3.csv")
    l_maps = pd.DataFrame()
    l_maps = pd.read_csv(f"{path}/hourly_decay_{decay}_rol_wind_size_{window_decay}.csv", sep=';')
    l_maps = l_maps.iloc[:,1:]
    l_maps['date'] = pd.to_datetime(l_maps['date'])
    l_maps.set_index('date', inplace = True)
    l_maps.rename(columns={"short_liquidation": "short_estimated_liquidation", "long_liquidation": "long_estimated_liquidation"}, inplace = True)
    
    return l_maps

l_maps = load_liquidation_map(0.1,720, path = '/opt/shared/pchen/liquidation_maps/data/bitfinex_distrib_long/bitfinex_2021/')

In [ ]:
l_maps

In [ ]:
# l_maps.iloc[:50].index

In [ ]:
# l_maps.iloc[-50:].index

[back to top](#outline_algos_framework) 

<div id='price_volume_profile'> </div> 

* ### price volume profile

In [ ]:
import pandas as pd
import numpy as np

def calculate_liquidity_map(df, price_bin_width=1000):
    """
    Calculates the trading volume within different price ranges.
    
    Parameters:
    - df: DataFrame with columns ['timestamp', 'price', 'amount']
    - price_bin_width: Width of the price range to consider
    
    Returns:
    A DataFrame containing the liquidity map
    """
    # Calculate bins for price ranges
    min_price = df['price'].min()
    max_price = df['price'].max()
    bins = np.arange(min_price, max_price + price_bin_width, price_bin_width)
    
    # Classify each transaction into a price bin and sum up the trading volume for each bin
    df['price_bin'] = pd.cut(df['price'], bins, labels=(bins[:-1] + bins[1:]) / 2)
    liquidity_map = df.groupby('price_bin')['amount'].sum().reset_index()
    
    return liquidity_map

# Calculate liquidity map with a price bin width of 100
liquidity_map = calculate_liquidity_map(df, price_bin_width=100)

print(liquidity_map)


[back to top](#outline_algos_framework)

<div id='dash_app'> </div> 

## dash app 

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

app = dash.Dash(__name__)

# Define your layout
app.layout = html.Div([
    dcc.Graph(id='live-plot'),
    dcc.Interval(id='graph-update', interval=1*1000, n_intervals=0)  # Update every 1 second
])

# Define callback to update graph
@app.callback(Output('live-plot', 'figure'),
              [Input('graph-update', 'n_intervals')])
def update_graph(n):
    # Your function to generate a figure
    fig = plot_framework(plot_requests=plot_requests,
                         prices=vwap_dict['BTC-TUSD']['vwap'],
                         framework_results=framework_results, 
                         downsample_n=20_000, 
                         ideal_bottoms=None, 
                         ideal_tops=None, )
    return fig




if __name__ == '__main__':
    app.run_server(debug=True, port=6969)


In [ ]:
plot_requests

[back to top](#outline_algos_framework)

<div id='live_dev_scratch_space'> </div> 


## continious live development scratch space



* ## adding leg group to positions table 

In [ ]:
now = datetime.datetime.utcnow()
leg_group_id = int(now.timestamp() * 1000)

In [ ]:
1688755358221  # bobs method

In [ ]:
def create_table(client):
    client.execute('''
        CREATE TABLE IF NOT EXISTS algos.TempPositions2 AS algos.Positions
    ''')
create_table(ch_client)

In [ ]:
cutoff_time = "2023-07-10 13:42:36"
query = f"""SELECT *
FROM algos.Positions
WHERE algo = 'prod_1____BTC_USDT_trades_only_data'     
ORDER BY timestamp desc;""" 
#     timestamp <= '{cutoff_time}'

# positions = ch_client.execute(query)
positions = ch_client.query_dataframe(query)

In [ ]:
positions['timestamp'] = pd.to_datetime(positions['timestamp'])
positions['leg_group_id'] = positions['timestamp'].astype(int) / 10**6
positions['leg_group_id'] = positions['leg_group_id'].astype(int)

In [ ]:
def insert_data(client, df):
    # Assuming that columns of df correspond to those in the table
    insert_query = 'INSERT INTO algos.Positions VALUES'

    values = df.to_dict('records')
    client.execute(insert_query, values)

# Call the function to insert data
insert_data(ch_client, positions)

In [ ]:
# cutoff_time = "2023-07-10 13:42:36"
# delete_query = f"""
# ALTER TABLE algos.Positions
# DELETE WHERE 
#     algo = 'prod_1____BTC_USDT_trades_only_data' AND
#     timestamp <= '{cutoff_time}';""" 

# # positions = ch_client.execute(query)
# num_deleted = ch_client.query_dataframe(delete_query)

* ## checking the mechanics of trading BTC with the "initial short" notional staying the same 

In [ ]:
asset_price = 30_000  
asset_price = 24_000
asset_price = 48_000
asset_price = 24_000

* #### with the portfolio (if it were trading actual long / short) 

In [ ]:
port_usd = 50
port_usd = port_usd * 1.2 # short (asset goes down by 20%) 
port_usd = port_usd * 2   # long  (and asset goes up by 100% )
port_usd = port_usd * 1.5 # short (asset goes down by 50% ) 
port_usd

* #### portfolio how it is  being ran
    * first, the actual spot part 

*  then the theoretical permenant short open on BTC for the same initial notional 

In [ ]:
# actual spot mechanics of trade ---- SPOT SIDE
spot_usd = 50
# spot_usd = spot_usd * 0.8 # short (asset goes down by 20%)   # commented out because there is no change (ALL IN USD during a short in the spot portfolio)
spot_usd = (2*spot_usd) * 2   # long  (and asset goes up by 100% )
# spot_usd = spot_usd * 1.5 # short (asset goes down by 50% )  # commented out because there is no change (ALL IN USD during a short in the spot portfolio)
spot_usd

In [ ]:
# actual spot mechanics of trade ---- SPOT SIDE
initial_fake_short_value = 50
fake_short_value = initial_fake_short_value * 1.2 # short (asset goes down by 20%)   # commented out because there is no change (ALL IN USD during a short in the spot portfolio)
fake_short_value = initial_fake_short_value * 0.4   # long  (and asset goes up by 100% )
fake_short_value = initial_fake_short_value * 1.2 # short (asset goes down by 50% )  # commented out because there is no change (ALL IN USD during a short in the spot portfolio)
fake_short_value


# \#
# \# 
# ###START DEV: Avellaneda Stoikov Market Making 

[back to top](#outline) 

<div id='avellaneda_stoikov_mm'> </div> 

* ### avellaneda stoikov market making metric ---- feature generation

#### definitions
* `s`: This is the current market mid-price of BTC. You would typically get this value from the exchange's API.
* `q`: This is the quantity of assets in the inventory of the base asset. It can be positive for long positions or negative for short positions. This would depend on your current inventory of BTC.
* `sigma`: This is the market volatility. You could calculate this using historical price data. For BTC, you would typically calculate this from historical BTC price data. Volatility changes with market conditions and should be updated regularly.
* `T`: This is the closing time, when the measurement period ends. This can be set based on your trading strategy. For example, if you plan to rebalance your inventory daily, you might set T=1.
* `t`: This is the current time. If T is normalized to 1, t is a time fraction.
* `gamma`: This is the inventory risk aversion parameter. This should be set based on your risk tolerance. A higher value implies that you are less willing to take on inventory risk.
* `kappa`: This is the order book liquidity parameter. This could be estimated from the order book depth, which can usually be obtained from the exchange's API. A higher value implies that the order book is denser.

In [ ]:
import numpy as np

class AvellanedaStoikov:
    def __init__(self, s, q, sigma, T, t, gamma, kappa):
        self.s = s
        self.q = q
        self.sigma = sigma
        self.T = T
        self.t = t
        self.gamma = gamma
        self.kappa = kappa

    def calculate_reservation_price(self):
        return self.s - self.gamma * self.sigma ** 2 * (self.T - self.t) * self.q

    def calculate_optimal_spread(self):
        term1 = self.sigma ** 2 * (self.T - self.t) + 2 / np.exp(self.kappa * (self.T - self.t)) - 2
        term2 = self.gamma * self.sigma ** 2 * (self.T - self.t)
        return np.sqrt(term1) + term2

    def calculate_bid_ask(self):
        res_price = self.calculate_reservation_price()
        optimal_spread = self.calculate_optimal_spread()

        bid_offer_price = res_price - optimal_spread / 2
        ask_offer_price = res_price + optimal_spread / 2

        return bid_offer_price, ask_offer_price


In [ ]:
prices.head()

In [ ]:
as_mm_df = pd.DataFrame(columns=['s', 'reserve_price', 'optimal_spread', 'q', 'sigma', 'T', 'gamma', 'kappa'], index=prices.index)
as_mm_df['s'] = prices['vwap']
as_mm_df['q'].iloc[0] = 0
as_mm_df['T'] = 1
as_mm_df['gamma'] = 1
as_mm_df['kappa'] = 1


as_mm_df.head(2) 

the iterative calculation of q 
* this is needed each iteration... for now we are going to have to get it by looking at the change of the price over a certain time period 

#### definitions
* `s`: This is the current market mid-price of BTC. You would typically get this value from the exchange's API.
* `q`: This is the quantity of assets in the inventory of the base asset. It can be positive for long positions or negative for short positions. This would depend on your current inventory of BTC.
* `sigma`: This is the market volatility. You could calculate this using historical price data. For BTC, you would typically calculate this from historical BTC price data. Volatility changes with market conditions and should be updated regularly.
* `T`: This is the closing time, when the measurement period ends. This can be set based on your trading strategy. For example, if you plan to rebalance your inventory daily, you might set T=1.
* `t`: This is the current time. If T is normalized to 1, t is a time fraction.
* `gamma`: This is the inventory risk aversion parameter. This should be set based on your risk tolerance. A higher value implies that you are less willing to take on inventory risk.
* `kappa`: This is the order book liquidity parameter. This could be estimated from the order book depth, which can usually be obtained from the exchange's API. A higher value implies that the order book is denser.

# ###END DEV: Avellaneda Stoikov Market Making 
# \#
# \# 

[back to top](#outline_algos_framework) 

<div id='copy_pasta'> </div> 

# copy pasta 
* #### no dev below this point

In [ ]:
# ### ONE LINE WRITE PICKLE
#
#
import pickle; fp = '/home/pboehringer/source/alpha-crypto/notebooks/data/prices_binance_btc_usdt_2017-08-18_2021-04-19.pickle';  pickle.dump(start_end_tiple_list, open(fp, "wb"))

  
        
# MULTI LINE 
#

# ### SAVING AND OPENING 
#
import pickle;
var_name = f"test_signal_dict____0_06_peaks____L1_loss"  #   <<<------- COPY PASTE DESIRED VARIABLE HERE 
fp = f"{params['dirs']['data_dir']}pickled_signal_dicts{var_name}.pickle"

# SAVING
pickle.dump(test_signal_dict____0_06_peaks____L1_loss,  #   <<<------- COPY PASTE DESIRED VARIABLE HERE
            open(fp, "wb")) 

# OPENING
with open(fp, 'rb') as handle:
     test_signal_dict____0_06_peaks____L1_loss = pickle.load(handle) #   <<<------- COPY PASTE DESIRED VARIABLE HERE

<div id='sql_console_copypasta'> </div>

# sql console

```sql
DESCRIBE amberdata.Trades

WITH
     '2018-01-01' as startdate,
     '2018-01-03' as enddate
SELECT *
FROM amberdata.Trades
WHERE
    exchange = 'binance'
    AND instrument='btc_usdt'
    AND date (timestamp) >= startdate
    AND date (timestamp) < enddate  -- not inclusive so it can be used to gather large date ranges in a rolling fashion
    ORDER BY timestamp

SELECT MAX(timestamp)
FROM amberdata.Trades
WHERE
    exchange = 'binance'
    AND instrument='btc_usdt'


SELECT MAX(timestamp)
FROM amberdata.Trades
WHERE
    exchange = 'binance'
    AND instrument='btc_usdt'
```

* ## copypasta: debubbing targets, could be useful later 

In [ ]:
import pickle; 
from algos.targets import (get_peak_and_valley_idxs_in_intervals, 
                            make_peaks_vallies_to_alternating_buy_sell_idxs,
                            get_datetimes_for_buy_sell_idxs,
                            make_targets_from_buy_sell_idxs)
from algos.plotting import (plot_price_with_buy_sell_times_on_price) 

fp = '/home/pboehringer/source/alpha-crypto/notebooks/data/debug_del_later____prices_error_on_target_making.pickle'; 
fp = '/home/pboehringer/source/alpha-crypto/notebooks/data/debug_del_later____prices_error_on_target_making2.pickle'; 

with open(fp, 'rb') as handle:
     test_prices = pickle.load(handle)

series=test_prices
min_peak_dist=25
peak_prominence=0.055
interval_len_in_steps=43200
smoothing_window=None

y_for_target_making_dtis, _ = make_targets(series=series,
                                           min_peak_dist=min_peak_dist,
                                           peak_prominence=peak_prominence,
                                           interval_len_in_steps=interval_len_in_steps,
                                           smoothing_window=None)


In [ ]:
# ### ENDING CELL: 
#
#










































